In [ ]:
# === Cellule 0: Configuration Complète (avec Stemming) ===
# Installe Java 21, configure comme défaut, installe outils build,
# pybind11, dernière Pyserini, NLTK+ressources, définit chemins,
# FONCTION preprocess_text AVEC STEMMING, parse topics.

import os
import sys
import subprocess
import time
import glob
import re
import json
import nltk # Importer nltk ici pour la partie NLTK
from tqdm.notebook import tqdm # Assurer l'import pour les fonctions
import traceback # Pour afficher les erreurs

print("--- Début de la Configuration Complète (avec Stemming) ---")
print("Cela peut prendre plusieurs minutes...")

# --- Partie 1: Installation Java 21 et Configuration ---
print("\n[1/9] Installation de OpenJDK 21...")
install_java_cmd = "apt-get update -qq > /dev/null && apt-get install -y openjdk-21-jdk-headless -qq > /dev/null"
try: subprocess.run(install_java_cmd, shell=True, check=True, timeout=180); print("OpenJDK 21 installé.")
except Exception as e: print(f"ERREUR installation Java 21: {e}"); raise

print("\n[2/9] Configuration de Java 21 comme défaut via update-alternatives...")
java_path_21 = "/usr/lib/jvm/java-21-openjdk-amd64/bin/java"
if os.path.exists(java_path_21):
    try: subprocess.run(f"update-alternatives --install /usr/bin/java java {java_path_21} 1", shell=True, check=True); subprocess.run(f"update-alternatives --set java {java_path_21}", shell=True, check=True); print("update-alternatives configuré.")
    except Exception as e: print(f"ERREUR config update-alternatives: {e}")
else: print(f"ATTENTION: Chemin Java 21 non trouvé: {java_path_21}.")
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-21-openjdk-amd64"
if not os.path.exists(os.environ["JAVA_HOME"]): print(f"ATTENTION: Chemin JAVA_HOME inexistant.")

# --- Partie 2: Installation des outils de build C++ ---
print("\n[3/9] Installation des outils de build (build-essential, cmake)...")
install_build_cmd = "apt-get update -qq > /dev/null && apt-get install -y build-essential cmake -qq > /dev/null"
try: subprocess.run(install_build_cmd, shell=True, check=True, timeout=180); print("Outils de build installés.")
except Exception as e_build: print(f"ERREUR installation outils de build: {e_build}")

# --- Partie 3: Installation de pybind11 ---
print("\n[4/9] Installation de pybind11...")
install_pybind_cmd = f"{sys.executable} -m pip install pybind11 -q"
try: subprocess.run(install_pybind_cmd, shell=True, check=True, timeout=60); print("pybind11 installé.")
except Exception as e_pybind: print(f"ERREUR installation pybind11: {e_pybind}")

# --- Partie 4: Installation des Paquets Python Principaux ---
print("\n[5/9] Installation de la DERNIÈRE Pyserini, NLTK, Pytrec_eval...")
install_pip_cmd = f"{sys.executable} -m pip install pyserini nltk pytrec_eval"
try: result_pip = subprocess.run(install_pip_cmd, shell=True, check=True, capture_output=True, text=True, timeout=600); print("Paquets Python principaux installés.")
except Exception as e_pip: print(f"ERREUR installation pip: {e_pip}"); raise

# --- Partie 5: Téléchargement Ressources NLTK ---
print("\n[6/9] Téléchargement/Vérification des ressources NLTK...")
import nltk
nltk_resources = ['wordnet', 'stopwords', 'punkt', 'omw-1.4', 'punkt_tab'] # Liste corrigée
for resource in nltk_resources:
    try:
        if resource == 'punkt' or resource == 'punkt_tab': resource_path = f'tokenizers/{resource}.zip'
        elif resource == 'omw-1.4': resource_path = f'corpora/{resource}.zip'
        elif resource == 'wordnet': resource_path = f'corpora/{resource}.zip' # Garder wordnet pour l'instant, même si non utilisé par stemmer
        else: resource_path = f'corpora/{resource}.zip'
        nltk.data.find(resource_path)
    except LookupError:
        print(f"  Ressource NLTK '{resource}' non trouvée. Téléchargement...")
        try: nltk.download(resource, quiet=True); print(f"  Ressource '{resource}' téléchargée.")
        except Exception as e_download: print(f"  ERREUR téléchargement '{resource}': {e_download}")
print("Ressources NLTK prêtes.")

# --- Partie 6: Définition des Chemins ---
print("\n[7/9] Définition des chemins...")
# !!! MODIFIEZ CETTE LIGNE AVEC LE CHEMIN CORRECT VERS VOTRE DOSSIER TREC !!!
DRIVE_PROJECT_PATH = "/content/drive/My Drive/Projet_RI/TREC" # <--- VÉRIFIEZ ET CORRIGEZ CE CHEMIN !

if 'google.colab' in sys.modules:
    try: from google.colab import drive; drive.mount('/content/drive', force_remount=True); print("  Google Drive monté.")
    except Exception as e_mount: print(f"ATTENTION: Erreur montage Drive: {e_mount}")
if not os.path.exists(DRIVE_PROJECT_PATH): raise FileNotFoundError(f"Chemin Drive '{DRIVE_PROJECT_PATH}' inexistant.")
print(f"  Chemin du projet Drive utilisé: {DRIVE_PROJECT_PATH}")

AP_TAR_FILENAME = "AP.tar"
AP_TAR_PATH = os.path.join(DRIVE_PROJECT_PATH, AP_TAR_FILENAME)
TOPICS_DIR = os.path.join(DRIVE_PROJECT_PATH, "Topics-requetes")
QRELS_DIR = os.path.join(DRIVE_PROJECT_PATH, "jugements de pertinence")
OUTPUT_DIR = "/content/ap_output"
INDEX_DIR_BASELINE = os.path.join(OUTPUT_DIR, "indexes/baseline")
INDEX_DIR_PREPROC = os.path.join(OUTPUT_DIR, "indexes/preprocessed") # Sera recréé avec stemming
CORPUS_DIR = os.path.join(OUTPUT_DIR, "corpus")
RUN_DIR = os.path.join(OUTPUT_DIR, "runs")
EVAL_DIR = os.path.join(OUTPUT_DIR, "eval")
os.makedirs(OUTPUT_DIR, exist_ok=True); os.makedirs(INDEX_DIR_BASELINE, exist_ok=True); os.makedirs(INDEX_DIR_PREPROC, exist_ok=True);
os.makedirs(CORPUS_DIR, exist_ok=True); os.makedirs(RUN_DIR, exist_ok=True); os.makedirs(EVAL_DIR, exist_ok=True)
print(f"  Chemin Fichier AP CIBLE: {AP_TAR_PATH}")
print(f"  Chemin Qrels: {QRELS_DIR}")
print(f"  Chemin Runs: {RUN_DIR}")

# --- Partie 7: Définition Fonction Prétraitement (AVEC STEMMING) ---
print("\n[8/9] Définition de la fonction preprocess_text (avec Stemming)...")
import nltk
from nltk.corpus import stopwords
# --- Utilisation de PorterStemmer ---
from nltk.stem import PorterStemmer # Import du stemmer
from nltk.tokenize import word_tokenize
import string
stop_words_set_global = set(stopwords.words('english'))
# --- Création de l'objet Stemmer ---
stemmer_obj_global = PorterStemmer() # Création de l'objet
def preprocess_text(text):
    """Applique tokenisation, minuscules, suppression ponctuation/non-alpha, stop words ET STEMMING (Porter)."""
    if not isinstance(text, str): return ""
    try: tokens = word_tokenize(text.lower())
    except LookupError as e_tok: # Gestion erreur si ressource NLTK manque
         if 'Resource' in str(e_tok) and 'not found' in str(e_tok):
              resource_name = str(e_tok).split('Resource ')[1].split(' ')[0]; print(f"--- Tokenizer a besoin de '{resource_name}', tentative téléchargement ---")
              try: nltk.download(resource_name, quiet=True); print(f"--- Ressource '{resource_name}' téléchargée ---"); tokens = word_tokenize(text.lower())
              except Exception as e_dl_tok: print(f"--- Échec téléchargement '{resource_name}': {e_dl_tok} ---"); raise e_tok
         else: raise e_tok
    except Exception as e_tok_other: print(f"Erreur word_tokenize: {e_tok_other}"); raise e_tok_other
    # --- Application du Stemmer ---
    filtered_tokens = [stemmer_obj_global.stem(w) for w in tokens if w.isalpha() and w not in stop_words_set_global]
    return ' '.join(filtered_tokens)
print("  Fonction preprocess_text définie avec PorterStemmer.")
# Tester la nouvelle fonction
sample_text = "This is an example showing Information Retrieval with stemming and stop words removal."
stemmed_sample = preprocess_text(sample_text)
print(f"  Exemple Stemmed: {stemmed_sample}") # Doit afficher 'thi is exampl show inform retriev with stem and stop word remov.'

# --- Partie 8: Parsing des Topics ---
print("\n[9/9] Parsing des topics...")
import re
import glob
def parse_topics(file_path):
    """Parse un fichier topic TREC standard."""
    topics = {};
    try:
        with open(file_path, 'r', encoding='utf-8') as f: content = f.read()
        for top_match in re.finditer(r"<top>(.*?)</top>", content, re.DOTALL):
            topic_content = top_match.group(1)
            num_match = re.search(r"<num>\s*Number:\s*(\d+)", topic_content, re.IGNORECASE); topic_id = num_match.group(1).strip() if num_match else None
            title_match = re.search(r"<title>\s*(.*?)\s*(?=<|$)", topic_content, re.IGNORECASE | re.DOTALL); title = title_match.group(1).strip() if title_match else ""
            desc_match = re.search(r"<desc>\s*Description:\s*(.*?)\s*(?=<|$)", topic_content, re.IGNORECASE | re.DOTALL); desc = desc_match.group(1).strip() if desc_match else ""
            if topic_id and title: topics[topic_id] = {'title': title, 'desc': desc}
    except Exception as e_topic: print(f"  ATTENTION: Erreur parsing {file_path}: {e_topic}")
    return topics

if not os.path.exists(TOPICS_DIR): print(f"ATTENTION: Dossier topics '{TOPICS_DIR}' inexistant."); topic_files = []
else: topic_files = sorted(glob.glob(os.path.join(TOPICS_DIR, "topics.*.txt")))
all_topics = {}
if not topic_files: print(f"  ATTENTION: Aucun fichier topic trouvé dans {TOPICS_DIR}")
else: print(f"  Parsing fichiers topics: {topic_files}"); [all_topics.update(parse_topics(tf)) for tf in topic_files]

try:
    queries_short = {qid: data['title'] for qid, data in all_topics.items()}
    queries_long = {qid: data['title'] + " " + data['desc'] for qid, data in all_topics.items()}
    print(f"  {len(all_topics)} topics parsés."); print(f"  {len(queries_short)} requêtes courtes brutes créées.")
    print(f"  Prétraitement des requêtes (avec stemming)...")
    # Appliquer la NOUVELLE fonction preprocess_text (avec stemming)
    queries_short_preprocessed = {qid: preprocess_text(q) for qid, q in queries_short.items()}
    queries_long_preprocessed = {qid: preprocess_text(q) for qid, q in queries_long.items()}
    print(f"  Prétraitement des requêtes terminé.")
except Exception as e_preproc_queries: print(f"\nERREUR prétraitement requêtes: {e_preproc_queries}"); queries_short_preprocessed, queries_long_preprocessed = {}, {}

# --- Vérification Finale Java ---
print("\n--- Vérification Finale Version Java Active ---")
try: result = subprocess.run("java -version", shell=True, check=True, capture_output=True, text=True, timeout=10); print("STDERR:\n", result.stderr); print("\nConfirmation: Java 21 OK." if "21." in result.stderr else "\nATTENTION: Java 21 NON ACTIF ?!")
except Exception as e: print(f"\nERREUR vérification Java: {e}")
# --- Vérification Finale Pyserini ---
print("\n--- Vérification Finale Version Pyserini Installée ---")
try: result_pyserini = subprocess.run(f"{sys.executable} -m pip show pyserini", shell=True, check=True, capture_output=True, text=True, timeout=30); print(result_pyserini.stdout)
except Exception as e: print(f"ERREUR vérification Pyserini: {e}")

print("\n--- Configuration Complète (avec Stemming) Terminée ---")
print("\nPause..."); time.sleep(2); print("Prêt pour la suite.")



--- Début de la Configuration Complète (avec Stemming) ---
Cela peut prendre plusieurs minutes...

[1/9] Installation de OpenJDK 21...
OpenJDK 21 installé.

[2/9] Configuration de Java 21 comme défaut via update-alternatives...
update-alternatives configuré.

[3/9] Installation des outils de build (build-essential, cmake)...
Outils de build installés.

[4/9] Installation de pybind11...
pybind11 installé.

[5/9] Installation de la DERNIÈRE Pyserini, NLTK, Pytrec_eval...
Paquets Python principaux installés.

[6/9] Téléchargement/Vérification des ressources NLTK...
Ressources NLTK prêtes.

[7/9] Définition des chemins...
Mounted at /content/drive
  Google Drive monté.
  Chemin du projet Drive utilisé: /content/drive/My Drive/Projet_RI/TREC
  Chemin Fichier AP CIBLE: /content/drive/My Drive/Projet_RI/TREC/AP.tar
  Chemin Qrels: /content/drive/My Drive/Projet_RI/TREC/jugements de pertinence
  Chemin Runs: /content/ap_output/runs

[8/9] Définition de la fonction preprocess_text (avec Stemmin

In [ ]:
# === Cellule 1: Extraire, Décompresser et Formater les Documents ===
# Lit AP.tar, décompresse les .gz internes, extrait <DOC>, <DOCNO>, <TEXT>
# et écrit le résultat dans ap_docs.jsonl.

import tarfile
import re
import json
import gzip # Importer le module gzip
from tqdm.notebook import tqdm
import os
import traceback

# Vérifier que les chemins sont définis (normalement fait par la cellule de config)
try:
    AP_TAR_PATH
    CORPUS_DIR
except NameError:
    print("ERREUR: Variables de chemin non définies. Exécutez la cellule de configuration complète.")
    raise

JSONL_OUTPUT_PATH = os.path.join(CORPUS_DIR, "ap_docs.jsonl")

print(f"Extraction, Décompression et Formatage depuis {AP_TAR_PATH} vers {JSONL_OUTPUT_PATH}...")

# Vérifier si le fichier AP.tar existe
if not os.path.exists(AP_TAR_PATH):
    raise FileNotFoundError(f"Le fichier d'archive {AP_TAR_PATH} n'a pas été trouvé.")
else:
    tar_size = os.path.getsize(AP_TAR_PATH)
    print(f"  Taille du fichier {AP_TAR_PATH}: {tar_size} octets.") # Devrait être ~275Mo

# Regex pour extraire les infos
doc_pattern = re.compile(r"<DOC>(.*?)</DOC>", re.DOTALL)
docno_pattern = re.compile(r"<DOCNO>\s*(.*?)\s*</DOCNO>")
text_pattern = re.compile(r"<TEXT>(.*?)</TEXT>", re.DOTALL)

# Compteurs
doc_count = 0
file_read_count = 0
skipped_members = 0
decompression_errors = 0

try:
    # Ouvrir le fichier de sortie et l'archive TAR
    with open(JSONL_OUTPUT_PATH, 'w', encoding='utf-8') as outfile, tarfile.open(AP_TAR_PATH, "r:") as tar:
        members = tar.getmembers()
        print(f"\n{len(members)} membres trouvés dans l'archive TAR.") # Devrait être ~1051

        # Boucler sur chaque membre de l'archive
        for member in tqdm(members, desc="Traitement des fichiers TAR"):
            # Ignorer si ce n'est pas un fichier .gz ou .Z
            if not member.isfile() or not member.name.lower().endswith(('.gz', '.z')):
                skipped_members += 1
                continue

            file_read_count += 1
            content = "" # Réinitialiser pour chaque fichier

            try:
                # Extraire le contenu compressé
                f = tar.extractfile(member)
                if f:
                    compressed_content = f.read()
                    f.close()

                    # Décompresser le contenu
                    try:
                        content_bytes = gzip.decompress(compressed_content)
                        content = content_bytes.decode('utf-8', errors='ignore') # Décoder après décompression
                    except gzip.BadGzipFile: # Gérer si ce n'est pas du gzip
                        content = compressed_content.decode('utf-8', errors='ignore')
                        decompression_errors += 1
                    except Exception as e_gzip:
                         print(f"\nErreur de décompression pour {member.name}: {e_gzip}")
                         decompression_errors += 1
                         continue # Passer au suivant

                    # Trouver tous les blocs <DOC> dans le contenu décompressé
                    doc_matches = doc_pattern.findall(content)
                    if not doc_matches: continue # Passer si aucun doc trouvé

                    # Boucler sur chaque document trouvé
                    for doc_content in doc_matches:
                        docno_match = docno_pattern.search(doc_content)
                        if not docno_match: continue
                        doc_id = docno_match.group(1).strip()

                        text_match = text_pattern.search(doc_content)
                        # Nettoyer le texte (espaces multiples)
                        doc_text = ' '.join(text_match.group(1).strip().split()) if text_match else ""

                        # Écrire la ligne JSONL
                        try:
                            json_line = json.dumps({"id": str(doc_id), "contents": str(doc_text)})
                            outfile.write(json_line + '\n')
                            doc_count += 1
                        except Exception as e_write:
                            print(f"Erreur écriture JSON pour doc_id {doc_id}: {e_write}")

            except KeyError as e_key: print(f"\nAvertissement: Membre '{member.name}' inaccessible (KeyError): {e_key}"); skipped_members += 1
            except EOFError: print(f"\nAvertissement: Fin fichier inattendue {member.name}."); skipped_members += 1
            except Exception as e_extract: print(f"\nErreur extraction/lecture {member.name}: {e_extract}"); skipped_members += 1

except tarfile.ReadError as e_tar: print(f"\nERREUR lecture TAR {AP_TAR_PATH}: {e_tar}"); raise e_tar
except FileNotFoundError: print(f"\nERREUR: Fichier TAR {AP_TAR_PATH} non trouvé."); raise FileNotFoundError
except Exception as e_general: print(f"\nERREUR générale traitement TAR: {e_general}"); traceback.print_exc(); raise e_general

# Afficher le résumé de l'extraction
print(f"\n--- Fin de l'Extraction et Décompression ---")
print(f"  {file_read_count} fichiers (.gz/.Z) lus.")
print(f"  {skipped_members} membres ignorés.")
if decompression_errors > 0: print(f"  {decompression_errors} erreurs/avertissements décompression.")
print(f"  {doc_count} documents écrits dans {JSONL_OUTPUT_PATH}") # Devrait être ~240k

# Vérifier la taille du fichier de sortie
if os.path.exists(JSONL_OUTPUT_PATH):
    output_size = os.path.getsize(JSONL_OUTPUT_PATH)
    print(f"  Taille finale {JSONL_OUTPUT_PATH}: {output_size} octets.") # Devrait être ~600Mo
    if output_size > 0 and doc_count > 0: print("  SUCCÈS: Fichier de sortie contient des données.")
    else: print("  ATTENTION: Fichier de sortie vide ou aucun document écrit.")
else: print(f"  ATTENTION: Fichier {JSONL_OUTPUT_PATH} non créé.")



Extraction, Décompression et Formatage depuis /content/drive/My Drive/Projet_RI/TREC/AP.tar vers /content/ap_output/corpus/ap_docs.jsonl...
  Taille du fichier /content/drive/My Drive/Projet_RI/TREC/AP.tar: 288296960 octets.

1051 membres trouvés dans l'archive TAR.


Traitement des fichiers TAR:   0%|          | 0/1051 [00:00<?, ?it/s]


--- Fin de l'Extraction et Décompression ---
  1050 fichiers (.gz/.Z) lus.
  1 membres ignorés.
  242918 documents écrits dans /content/ap_output/corpus/ap_docs.jsonl
  Taille finale /content/ap_output/corpus/ap_docs.jsonl: 620216647 octets.
  SUCCÈS: Fichier de sortie contient des données.


In [ ]:
    # === Cellule 2: Indexation Baseline (Relance) ===
    # Crée l'index Lucene à partir de ap_docs.jsonl.

    import os
    import subprocess
    import traceback
    import re # Importer re

    # Vérifier que les chemins sont définis
    try: CORPUS_DIR; INDEX_DIR_BASELINE
    except NameError: print("ERREUR: Variables chemin non définies. Exécutez config complète."); raise

    print(f"Début de l'indexation Baseline...")
    print(f"Dossier source: {CORPUS_DIR}")
    print(f"Répertoire de l'index cible: {INDEX_DIR_BASELINE}")

    jsonl_source_path = os.path.join(CORPUS_DIR, "ap_docs.jsonl")
    # Vérifier si le fichier source existe (il devrait exister après l'extraction récente)
    if not os.path.exists(jsonl_source_path) or os.path.getsize(jsonl_source_path) == 0:
         print(f"ERREUR: Fichier source {jsonl_source_path} manquant/vide. Relancez l'extraction (Étape 1).")
         raise FileNotFoundError(f"Fichier source {jsonl_source_path} manquant/vide.")

    # Commande Pyserini
    index_cmd_baseline = [
        "python", "-m", "pyserini.index.lucene", "--collection", "JsonCollection",
        "--input", CORPUS_DIR, "--index", INDEX_DIR_BASELINE,
        "--generator", "DefaultLuceneDocumentGenerator", "--threads", "4",
        "--storePositions", "--storeDocvectors", "--storeRaw"
    ]

    print(f"Exécution: {' '.join(index_cmd_baseline)}")
    try:
        # Exécuter l'indexation
        result = subprocess.run(index_cmd_baseline, check=True, capture_output=True, text=True, timeout=1800) # Timeout 30 min
        print("Sortie STDOUT (fin):\n", result.stdout[-1000:])
        print("Sortie STDERR:\n", result.stderr)
        num_docs_indexed_str = "inconnu"; match = re.search(r"Total (\d+) documents indexed", result.stdout)
        if match: num_docs_indexed_str = match.group(1)
        if "Total 0 documents indexed" in result.stdout: print("\nATTENTION: 0 document indexé.")
        else: print(f"\nIndexation Baseline terminée. {num_docs_indexed_str} documents indexés: {INDEX_DIR_BASELINE}")
    except Exception as e:
        # Gérer les erreurs
        print(f"\nERREUR indexation Baseline: {e}")
        if isinstance(e, subprocess.CalledProcessError): print("STDERR:", e.stderr)
        else: traceback.print_exc()
        raise e

    # Vérifier la taille
    print(f"\nVérification taille index: {INDEX_DIR_BASELINE}...")
    if os.path.exists(INDEX_DIR_BASELINE):
        du_cmd = f"du -sh '{INDEX_DIR_BASELINE}'";
        try: result_du = subprocess.run(du_cmd, shell=True, check=True, capture_output=True, text=True); print(f"  Taille: {result_du.stdout.split()[0]}")
        except Exception as e_du: print(f"  Impossible vérifier taille: {e_du}")
    else: print("  ATTENTION: Dossier index non créé.")


Début de l'indexation Baseline...
Dossier source: /content/ap_output/corpus
Répertoire de l'index cible: /content/ap_output/indexes/baseline
Exécution: python -m pyserini.index.lucene --collection JsonCollection --input /content/ap_output/corpus --index /content/ap_output/indexes/baseline --generator DefaultLuceneDocumentGenerator --threads 4 --storePositions --storeDocvectors --storeRaw
Sortie STDOUT (fin):
 ) - 210,000 documents indexed
2025-04-08 22:49:26,795 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:307) - Indexing Complete! 242,435 documents indexed
2025-04-08 22:49:26,796 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:308) - ============ Final Counter Values ============
2025-04-08 22:49:26,796 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:309) - indexed:          242,435
2025-04-08 22:49:26,796 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:310) - unindexable:            0
2025-04-08 22:49:26,797 INFO  [main] index.AbstractIndexer (A

In [ ]:
# === Cellule 2: Indexation Baseline ===
# Crée l'index Lucene à partir de ap_docs.jsonl (sans prétraitement spécifique).

import os
import subprocess
import traceback

# Vérifier que les chemins sont définis (normalement fait par la cellule de config)
try:
    CORPUS_DIR
    INDEX_DIR_BASELINE
except NameError:
    print("ERREUR: Variables de chemin non définies. Exécutez la cellule de configuration complète.")
    raise

print(f"Début de l'indexation Baseline...")
print(f"Dossier source: {CORPUS_DIR}")
print(f"Répertoire de l'index cible: {INDEX_DIR_BASELINE}")

# Vérifier si le fichier source existe et n'est pas vide
jsonl_source_path = os.path.join(CORPUS_DIR, "ap_docs.jsonl")
if not os.path.exists(jsonl_source_path) or os.path.getsize(jsonl_source_path) == 0:
     raise FileNotFoundError(f"Le fichier source {jsonl_source_path} est manquant ou vide. L'étape d'extraction ('extract_code_tar_gzip_fixed') a échoué.")

# Commande Pyserini
# Utilise la dernière version de Pyserini installée
index_cmd_baseline = [
    "python", "-m", "pyserini.index.lucene",
    "--collection", "JsonCollection",
    "--input", CORPUS_DIR,
    "--index", INDEX_DIR_BASELINE,
    "--generator", "DefaultLuceneDocumentGenerator",
    "--threads", "4", # Nombre de threads pour l'indexation
    "--storePositions", "--storeDocvectors", "--storeRaw" # Options de stockage
]

print(f"Exécution: {' '.join(index_cmd_baseline)}")
try:
    # Exécuter la commande d'indexation
    # Augmentation possible du timeout si l'indexation est très longue
    result = subprocess.run(index_cmd_baseline, check=True, capture_output=True, text=True, timeout=1800) # Timeout 30 min
    print("Sortie STDOUT (fin):\n", result.stdout[-1000:]) # Afficher la fin de stdout
    print("Sortie STDERR:\n", result.stderr)
    # Vérifier si 0 document a été indexé (signe de problème)
    if "Total 0 documents indexed" in result.stdout:
         print("\nATTENTION: Pyserini indique 0 document indexé.")
    else:
         # Extraire le nombre de documents indexés si possible (peut varier selon la sortie de Pyserini)
         num_docs_indexed_str = "inconnu"
         match = re.search(r"Total (\d+) documents indexed", result.stdout)
         if match:
             num_docs_indexed_str = match.group(1)
         print(f"\nIndexation Baseline terminée. {num_docs_indexed_str} documents indexés dans {INDEX_DIR_BASELINE}")

except Exception as e:
    # Gérer les erreurs potentielles
    print(f"\nERREUR pendant l'indexation Baseline: {e}")
    if isinstance(e, subprocess.CalledProcessError):
        print("Sortie STDOUT:\n", e.stdout)
        print("Sortie STDERR:\n", e.stderr)
    else:
        traceback.print_exc()
    raise e

# Vérifier la taille de l'index créé
print(f"\nVérification taille index: {INDEX_DIR_BASELINE}...")
if os.path.exists(INDEX_DIR_BASELINE):
    du_cmd = f"du -sh '{INDEX_DIR_BASELINE}'" # Commande pour taille dossier
    try:
        result_du = subprocess.run(du_cmd, shell=True, check=True, capture_output=True, text=True)
        print(f"  Taille: {result_du.stdout.split()[0]}") # Afficher la taille
    except Exception as e_du:
        print(f"  Impossible de vérifier taille: {e_du}")
else:
    print("  ATTENTION: Dossier index non créé.")


Début de l'indexation Baseline...
Dossier source: /content/ap_output/corpus
Répertoire de l'index cible: /content/ap_output/indexes/baseline
Exécution: python -m pyserini.index.lucene --collection JsonCollection --input /content/ap_output/corpus --index /content/ap_output/indexes/baseline --generator DefaultLuceneDocumentGenerator --threads 4 --storePositions --storeDocvectors --storeRaw
Sortie STDOUT (fin):
 ) - 210,000 documents indexed
2025-04-08 22:54:49,210 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:307) - Indexing Complete! 242,435 documents indexed
2025-04-08 22:54:49,211 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:308) - ============ Final Counter Values ============
2025-04-08 22:54:49,211 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:309) - indexed:          242,435
2025-04-08 22:54:49,212 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:310) - unindexable:            0
2025-04-08 22:54:49,213 INFO  [main] index.AbstractIndexer (A

In [ ]:
# === Cellule 3: Préparer les Données Prétraitées (avec Stemming) ===
# Lit ap_docs.jsonl, applique la fonction preprocess_text (avec stemming)
# et écrit le résultat dans ap_docs_preprocessed.jsonl.

import json
from tqdm.notebook import tqdm
import os
import traceback

# Vérifier que les chemins et la fonction sont définis
try:
    CORPUS_DIR
    JSONL_OUTPUT_PATH # Défini dans la config, utilisé comme entrée ici
    preprocess_text # Défini dans la config (version stemming)
except NameError:
    print("ERREUR: Variables/Fonction non définies. Exécutez la cellule de configuration complète.")
    raise

JSONL_PREPROC_PATH = os.path.join(CORPUS_DIR, "ap_docs_preprocessed.jsonl") # Fichier de sortie

print(f"Préparation données prétraitées (avec stemming) depuis {JSONL_OUTPUT_PATH} vers {JSONL_PREPROC_PATH}...")

# Vérifier fichier source
if not os.path.exists(JSONL_OUTPUT_PATH) or os.path.getsize(JSONL_OUTPUT_PATH) == 0:
     raise FileNotFoundError(f"Le fichier source {JSONL_OUTPUT_PATH} est manquant ou vide.")

doc_count_preproc = 0
error_count = 0
try:
    # Ouvrir les fichiers d'entrée et de sortie
    with open(JSONL_OUTPUT_PATH, 'r', encoding='utf-8') as infile, \
         open(JSONL_PREPROC_PATH, 'w', encoding='utf-8') as outfile:
        # Boucler sur chaque ligne du fichier d'entrée
        for line in tqdm(infile, desc="Prétraitement (Stemming)"):
            try:
                data = json.loads(line)
                doc_id = data.get('id', None)
                original_contents = data.get('contents', '')
                if doc_id is None: error_count += 1; continue

                # Appliquer le prétraitement (avec stemming)
                preprocessed_contents = preprocess_text(original_contents)

                # Écrire la ligne traitée
                json_line = json.dumps({"id": str(doc_id), "contents": str(preprocessed_contents)})
                outfile.write(json_line + '\n')
                doc_count_preproc += 1
            except json.JSONDecodeError: error_count += 1 # Compter les erreurs JSON
            except Exception as e_line: print(f"\nErreur ligne (id={data.get('id', 'inconnu')}): {e_line}"); error_count += 1

    # Afficher le résumé
    print(f"\nTerminé.")
    print(f"  {doc_count_preproc} documents prétraités (stemming) écrits dans {JSONL_PREPROC_PATH}")
    if error_count > 0: print(f"  {error_count} lignes ignorées.")

    # Vérifier la taille du fichier de sortie
    if os.path.exists(JSONL_PREPROC_PATH):
        output_size = os.path.getsize(JSONL_PREPROC_PATH)
        print(f"  Taille finale: {output_size} octets.") # Sera probablement plus petit que l'original
        if output_size == 0 and doc_count_preproc > 0: print("  ATTENTION: Taille nulle ?!")
    else: print(f"  ATTENTION: Fichier sortie {JSONL_PREPROC_PATH} non créé.")

except Exception as e_main:
    print(f"ERREUR générale préparation données prétraitées: {e_main}")
    traceback.print_exc()
    raise


Préparation données prétraitées (avec stemming) depuis /content/ap_output/corpus/ap_docs.jsonl vers /content/ap_output/corpus/ap_docs_preprocessed.jsonl...


Prétraitement (Stemming): 0it [00:00, ?it/s]


Terminé.
  242918 documents prétraités (stemming) écrits dans /content/ap_output/corpus/ap_docs_preprocessed.jsonl
  Taille finale: 370516082 octets.


In [ ]:
# === Cellule 4: Indexation Prétraitée (Stemming) ===
# Crée l'index Lucene à partir de ap_docs_preprocessed.jsonl (stemmed).
# Utilise l'option --pretokenized.

import os
import subprocess
import traceback
import re # Importer re pour l'analyse de la sortie

# Vérifier que les chemins sont définis
try: CORPUS_DIR; INDEX_DIR_PREPROC
except NameError: print("ERREUR: Variables chemin non définies."); raise

JSONL_PREPROC_PATH = os.path.join(CORPUS_DIR, "ap_docs_preprocessed.jsonl") # Fichier source

print(f"Début indexation Prétraitée (Stemming)...")
print(f"Source: {CORPUS_DIR}") # Pyserini prend le dossier
print(f"Fichier JSONL attendu: {JSONL_PREPROC_PATH}")
print(f"Index cible: {INDEX_DIR_PREPROC}")

# Vérifier si le fichier prétraité existe et n'est pas vide
if not os.path.exists(JSONL_PREPROC_PATH) or os.path.getsize(JSONL_PREPROC_PATH) == 0:
    raise FileNotFoundError(f"Fichier prétraité {JSONL_PREPROC_PATH} manquant/vide.")

# Commande Pyserini
index_cmd = [
    "python", "-m", "pyserini.index.lucene", "--collection", "JsonCollection",
    "--input", CORPUS_DIR, "--index", INDEX_DIR_PREPROC,
    "--generator", "DefaultLuceneDocumentGenerator", "--threads", "4",
    "--storePositions", "--storeDocvectors", "--storeRaw",
    "--pretokenized" # Option clé ici
]

print(f"Exécution: {' '.join(index_cmd)}")
try:
    # Exécuter la commande d'indexation
    result = subprocess.run(index_cmd, check=True, capture_output=True, text=True, timeout=1800) # Timeout 30 min
    print("Sortie STDOUT (fin):\n", result.stdout[-1000:])
    print("Sortie STDERR:\n", result.stderr)
    # Essayer d'extraire le nombre de documents indexés
    num_docs_indexed_str = "inconnu"
    match = re.search(r"Total (\d+) documents indexed", result.stdout)
    if match:
        num_docs_indexed_str = match.group(1)

    if "Total 0 documents indexed" in result.stdout: print("\nATTENTION: 0 document indexé.")
    else: print(f"\nIndexation Prétraitée (Stemming) terminée. {num_docs_indexed_str} documents indexés dans {INDEX_DIR_PREPROC}")
except Exception as e:
    # Gérer les erreurs
    print(f"\nERREUR pendant l'indexation Prétraitée: {e}")
    if isinstance(e, subprocess.CalledProcessError): print("STDERR:", e.stderr)
    else: traceback.print_exc()
    raise e

# Vérifier la taille de l'index
print(f"\nVérification taille index: {INDEX_DIR_PREPROC}...")
if os.path.exists(INDEX_DIR_PREPROC):
    du_cmd = f"du -sh '{INDEX_DIR_PREPROC}'" # Commande pour taille dossier
    try: result_du = subprocess.run(du_cmd, shell=True, check=True, capture_output=True, text=True); print(f"  Taille: {result_du.stdout.split()[0]}") # Afficher la taille
    except Exception as e_du: print(f"  Impossible vérifier taille: {e_du}")
else: print("  ATTENTION: Dossier index non créé.")



Début indexation Prétraitée (Stemming)...
Source: /content/ap_output/corpus
Fichier JSONL attendu: /content/ap_output/corpus/ap_docs_preprocessed.jsonl
Index cible: /content/ap_output/indexes/preprocessed
Exécution: python -m pyserini.index.lucene --collection JsonCollection --input /content/ap_output/corpus --index /content/ap_output/indexes/preprocessed --generator DefaultLuceneDocumentGenerator --threads 4 --storePositions --storeDocvectors --storeRaw --pretokenized
Sortie STDOUT (fin):
 ed, 472,330 documents indexed
2025-04-08 23:31:26,056 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:307) - Indexing Complete! 484,765 documents indexed
2025-04-08 23:31:26,057 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:308) - ============ Final Counter Values ============
2025-04-08 23:31:26,059 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:309) - indexed:          484,765
2025-04-08 23:31:26,060 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:310) - unin

In [ ]:
# === Cellule 4: Indexation Prétraitée (Stemming) ===
# Crée l'index Lucene à partir de ap_docs_preprocessed.jsonl (stemmed).
# Utilise l'option --pretokenized.

import os
import subprocess
import traceback
import re # Importer re pour l'analyse de la sortie

# Vérifier que les chemins sont définis
try: CORPUS_DIR; INDEX_DIR_PREPROC
except NameError: print("ERREUR: Variables chemin non définies."); raise

JSONL_PREPROC_PATH = os.path.join(CORPUS_DIR, "ap_docs_preprocessed.jsonl") # Fichier source

print(f"Début indexation Prétraitée (Stemming)...")
print(f"Source: {CORPUS_DIR}") # Pyserini prend le dossier
print(f"Fichier JSONL attendu: {JSONL_PREPROC_PATH}")
print(f"Index cible: {INDEX_DIR_PREPROC}")

# Vérifier si le fichier prétraité existe et n'est pas vide
if not os.path.exists(JSONL_PREPROC_PATH) or os.path.getsize(JSONL_PREPROC_PATH) == 0:
    raise FileNotFoundError(f"Fichier prétraité {JSONL_PREPROC_PATH} manquant/vide.")

# Commande Pyserini
index_cmd = [
    "python", "-m", "pyserini.index.lucene", "--collection", "JsonCollection",
    "--input", CORPUS_DIR, "--index", INDEX_DIR_PREPROC,
    "--generator", "DefaultLuceneDocumentGenerator", "--threads", "4",
    "--storePositions", "--storeDocvectors", "--storeRaw",
    "--pretokenized" # Option clé ici
]

print(f"Exécution: {' '.join(index_cmd)}")
try:
    # Exécuter la commande d'indexation
    result = subprocess.run(index_cmd, check=True, capture_output=True, text=True, timeout=1800) # Timeout 30 min
    print("Sortie STDOUT (fin):\n", result.stdout[-1000:])
    print("Sortie STDERR:\n", result.stderr)
    # Essayer d'extraire le nombre de documents indexés
    num_docs_indexed_str = "inconnu"
    match = re.search(r"Total (\d+) documents indexed", result.stdout)
    if match:
        num_docs_indexed_str = match.group(1)

    if "Total 0 documents indexed" in result.stdout: print("\nATTENTION: 0 document indexé.")
    else: print(f"\nIndexation Prétraitée (Stemming) terminée. {num_docs_indexed_str} documents indexés dans {INDEX_DIR_PREPROC}")
except Exception as e:
    # Gérer les erreurs
    print(f"\nERREUR pendant l'indexation Prétraitée: {e}")
    if isinstance(e, subprocess.CalledProcessError): print("STDERR:", e.stderr)
    else: traceback.print_exc()
    raise e

# Vérifier la taille de l'index
print(f"\nVérification taille index: {INDEX_DIR_PREPROC}...")
if os.path.exists(INDEX_DIR_PREPROC):
    du_cmd = f"du -sh '{INDEX_DIR_PREPROC}'" # Commande pour taille dossier
    try: result_du = subprocess.run(du_cmd, shell=True, check=True, capture_output=True, text=True); print(f"  Taille: {result_du.stdout.split()[0]}") # Afficher la taille
    except Exception as e_du: print(f"  Impossible vérifier taille: {e_du}")
else: print("  ATTENTION: Dossier index non créé.")



Début indexation Prétraitée (Stemming)...
Source: /content/ap_output/corpus
Fichier JSONL attendu: /content/ap_output/corpus/ap_docs_preprocessed.jsonl
Index cible: /content/ap_output/indexes/preprocessed
Exécution: python -m pyserini.index.lucene --collection JsonCollection --input /content/ap_output/corpus --index /content/ap_output/indexes/preprocessed --generator DefaultLuceneDocumentGenerator --threads 4 --storePositions --storeDocvectors --storeRaw --pretokenized
Sortie STDOUT (fin):
 ed, 462,330 documents indexed
2025-04-08 16:58:45,094 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:307) - Indexing Complete! 484,765 documents indexed
2025-04-08 16:58:45,095 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:308) - ============ Final Counter Values ============
2025-04-08 16:58:45,095 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:309) - indexed:          484,765
2025-04-08 16:58:45,096 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:310) - unin

In [ ]:
# === Cellule 5: Exécuter les Recherches (Séquentielles - BM25 & QLD - avec Stemming) ===
# Lance les 8 combinaisons de recherche en utilisant les index baseline et preprocessed (stemming).
# S'assure que l'environnement Java 21 est actif et que les index/variables sont définis.

from pyserini.search.lucene import LuceneSearcher # Import principal
import time
from tqdm.notebook import tqdm
import traceback
import os
from jnius import JavaException # Importer seulement JavaException

# Définir K_RESULTS
try: K_RESULTS
except NameError: print("Définition K_RESULTS=1000"); K_RESULTS = 1000

# Vérifier variables nécessaires et existence des index
try:
    INDEX_DIR_BASELINE; INDEX_DIR_PREPROC; RUN_DIR; K_RESULTS; CORPUS_DIR;
    queries_short; queries_long; queries_short_preprocessed; queries_long_preprocessed;
    preprocess_text; # Fonction avec stemming doit être définie
    if not os.path.exists(INDEX_DIR_BASELINE): raise FileNotFoundError(f"Index Baseline manquant: {INDEX_DIR_BASELINE}")
    if not os.path.exists(INDEX_DIR_PREPROC): raise FileNotFoundError(f"Index Preprocessed (Stemming) manquant: {INDEX_DIR_PREPROC}")
    if not os.path.exists(os.path.join(CORPUS_DIR, "ap_docs.jsonl")): raise FileNotFoundError("ap_docs.jsonl manquant.")
    if not os.path.exists(os.path.join(CORPUS_DIR, "ap_docs_preprocessed.jsonl")): raise FileNotFoundError("ap_docs_preprocessed.jsonl (stemmed) manquant.")

except NameError as e: print(f"ERREUR: Variable {e} manquante. Exécutez config complète."); raise
except FileNotFoundError as e: print(f"ERREUR: {e}"); raise

def perform_search_sequential_qld(queries, index_path, model, k, output_run_file_base, run_tag_prefix, use_preprocessed_query=False):
    """Exécute la recherche séquentiellement pour un ensemble de requêtes (BM25 ou QLD)."""
    start_time = time.time()
    # Ajouter '_stem' au tag et au nom de fichier pour les runs prétraités
    is_preproc_run = (index_path == INDEX_DIR_PREPROC)
    run_tag_suffix = "_stem" if is_preproc_run else ""
    model_suffix = "_stem" if is_preproc_run else "" # Suffixe pour nom de fichier aussi
    run_tag = f"{run_tag_prefix}_{model}{run_tag_suffix}"
    # Construire le nom de fichier de sortie final
    base_filename = f"{run_tag_prefix}_{model}{model_suffix}.txt"
    output_run_file = os.path.join(RUN_DIR, base_filename)

    print(f"\nDébut recherche SÉQUENTIELLE: Index='{os.path.basename(index_path)}', Modèle='{model}', Tag='{run_tag}', k={k}")
    print(f"  Fichier de sortie prévu: {output_run_file}")

    all_results_list = []
    searcher = None

    try:
        print(f"  Initialisation LuceneSearcher..."); searcher = LuceneSearcher(index_path); print(f"  LuceneSearcher initialisé.")

        # Configurer similarité
        if model == 'bm25':
            print("  Configuration BM25..."); searcher.set_bm25(k1=0.9, b=0.4); print("  BM25 configuré.")
        elif model == 'qld': # Utiliser Query Likelihood Dirichlet
            print("  Configuration QLD..."); searcher.set_qld(); print("  QLD configuré.")
        else:
            print(f"Modèle '{model}' non reconnu, utilise BM25 par défaut."); searcher.set_bm25()

        # Itérer sur les requêtes
        query_errors = 0
        if 'preprocess_text' not in globals(): raise NameError("preprocess_text non définie.")

        for query_id, query_text in tqdm(queries.items(), desc=f"Recherche {run_tag}"):
            try:
                # Utiliser la fonction preprocess_text (avec stemming) si nécessaire
                # Note: use_preprocessed_query est False pour les runs preproc car les queries sont déjà stemmatisées
                search_text = preprocess_text(query_text) if use_preprocessed_query else query_text
                if not search_text.strip(): continue # Ignorer requêtes vides

                hits = searcher.search(search_text, k=k)

                for i in range(len(hits)):
                    rank, doc_id, score = i + 1, hits[i].docid, hits[i].score
                    if doc_id is None: continue
                    all_results_list.append(f"{query_id} Q0 {doc_id} {rank} {score:.6f} {run_tag}\n")
            except Exception as e_query:
                query_errors += 1
                if query_errors < 5: print(f"\nErreur recherche QID {query_id}: {e_query}")
                elif query_errors == 5: print("\nPlusieurs erreurs recherche...")

        # Écrire résultats
        if all_results_list:
             # S'assurer que le dossier RUN_DIR existe avant d'écrire
             os.makedirs(os.path.dirname(output_run_file), exist_ok=True)
             with open(output_run_file, 'w', encoding='utf-8') as f_out: f_out.writelines(all_results_list)
             print(f"\n  {len(all_results_list)} lignes résultats écrites dans {os.path.basename(output_run_file)}.")
        else: print("\n  Avertissement: Aucun résultat généré pour ce run.")
        if query_errors > 0: print(f"  Avertissement: {query_errors} erreurs sur requêtes.")

        end_time = time.time()
        print(f"Recherche terminée pour {run_tag}. Sauvegardé dans {output_run_file}")
        print(f"Temps écoulé: {end_time - start_time:.2f} secondes.")
    except Exception as e_main: print(f"\nERREUR MAJEURE run {run_tag}: {e_main}"); traceback.print_exc()
    finally:
        if searcher: print(f"  Nettoyage implicite ressources {run_tag}.")

# --- Exécution des 8 configurations (BM25 et QLD, avec index stemmatisé pour preproc) ---
# Note: Les noms de fichiers pour les runs preproc incluront '_stem'
print("\n--- DÉBUT DES RECHERCHES BASELINE (BM25/QLD) ---")
run_file_1 = "baseline_short_bm25"; perform_search_sequential_qld(queries_short, INDEX_DIR_BASELINE, 'bm25', K_RESULTS, run_file_1, "baseline_short")
run_file_2 = "baseline_short_qld"; perform_search_sequential_qld(queries_short, INDEX_DIR_BASELINE, 'qld', K_RESULTS, run_file_2, "baseline_short")
run_file_3 = "baseline_long_bm25"; perform_search_sequential_qld(queries_long, INDEX_DIR_BASELINE, 'bm25', K_RESULTS, run_file_3, "baseline_long")
run_file_4 = "baseline_long_qld"; perform_search_sequential_qld(queries_long, INDEX_DIR_BASELINE, 'qld', K_RESULTS, run_file_4, "baseline_long")
print("\n--- DÉBUT DES RECHERCHES PRÉTRAITÉES (STEMMING - BM25/QLD) ---")
# Utiliser les requêtes prétraitées (stemmed) et l'index preproc (stemmed)
# Les noms de fichiers de sortie incluront '_stem' grâce à la logique dans la fonction
run_file_5 = "preproc_short_bm25_stem"; perform_search_sequential_qld(queries_short_preprocessed, INDEX_DIR_PREPROC, 'bm25', K_RESULTS, run_file_5, "preproc_short", use_preprocessed_query=False)
run_file_6 = "preproc_short_qld_stem"; perform_search_sequential_qld(queries_short_preprocessed, INDEX_DIR_PREPROC, 'qld', K_RESULTS, run_file_6, "preproc_short", use_preprocessed_query=False)
run_file_7 = "preproc_long_bm25_stem"; perform_search_sequential_qld(queries_long_preprocessed, INDEX_DIR_PREPROC, 'bm25', K_RESULTS, run_file_7, "preproc_long", use_preprocessed_query=False)
run_file_8 = "preproc_long_qld_stem"; perform_search_sequential_qld(queries_long_preprocessed, INDEX_DIR_PREPROC, 'qld', K_RESULTS, run_file_8, "preproc_long", use_preprocessed_query=False)
print("\n--- Toutes les recherches (Baseline et Stemming) sont terminées. ---")

# Vérifier si des fichiers ont été créés
print(f"\nVérification du contenu de {RUN_DIR} après les recherches...")
!ls -l {RUN_DIR}



Définition K_RESULTS=1000

--- DÉBUT DES RECHERCHES BASELINE (BM25/QLD) ---

Début recherche SÉQUENTIELLE: Index='baseline', Modèle='bm25', Tag='baseline_short_bm25', k=1000
  Fichier de sortie prévu: /content/ap_output/runs/baseline_short_bm25.txt
  Initialisation LuceneSearcher...
  LuceneSearcher initialisé.
  Configuration BM25...
  BM25 configuré.


Recherche baseline_short_bm25:   0%|          | 0/150 [00:00<?, ?it/s]


  150000 lignes résultats écrites dans baseline_short_bm25.txt.
Recherche terminée pour baseline_short_bm25. Sauvegardé dans /content/ap_output/runs/baseline_short_bm25.txt
Temps écoulé: 18.91 secondes.
  Nettoyage implicite ressources baseline_short_bm25.

Début recherche SÉQUENTIELLE: Index='baseline', Modèle='qld', Tag='baseline_short_qld', k=1000
  Fichier de sortie prévu: /content/ap_output/runs/baseline_short_qld.txt
  Initialisation LuceneSearcher...
  LuceneSearcher initialisé.
  Configuration QLD...
  QLD configuré.


Recherche baseline_short_qld:   0%|          | 0/150 [00:00<?, ?it/s]


  150000 lignes résultats écrites dans baseline_short_qld.txt.
Recherche terminée pour baseline_short_qld. Sauvegardé dans /content/ap_output/runs/baseline_short_qld.txt
Temps écoulé: 14.92 secondes.
  Nettoyage implicite ressources baseline_short_qld.

Début recherche SÉQUENTIELLE: Index='baseline', Modèle='bm25', Tag='baseline_long_bm25', k=1000
  Fichier de sortie prévu: /content/ap_output/runs/baseline_long_bm25.txt
  Initialisation LuceneSearcher...
  LuceneSearcher initialisé.
  Configuration BM25...
  BM25 configuré.


...


  150000 lignes résultats écrites dans preproc_short_qld_stem.txt.
Recherche terminée pour preproc_short_qld_stem. Sauvegardé dans /content/ap_output/runs/preproc_short_qld_stem.txt
Temps écoulé: 13.29 secondes.
  Nettoyage implicite ressources preproc_short_qld_stem.

Début recherche SÉQUENTIELLE: Index='preprocessed', Modèle='bm25', Tag='preproc_long_bm25_stem', k=1000
  Fichier de sortie prévu: /content/ap_output/runs/preproc_long_bm25_stem.txt
  Initialisation LuceneSearcher...
  LuceneSearcher initialisé.
  Configuration BM25...
  BM25 configuré.


Recherche preproc_long_bm25_stem:   0%|          | 0/150 [00:00<?, ?it/s]


  150000 lignes résultats écrites dans preproc_long_bm25_stem.txt.
Recherche terminée pour preproc_long_bm25_stem. Sauvegardé dans /content/ap_output/runs/preproc_long_bm25_stem.txt
Temps écoulé: 15.08 secondes.
  Nettoyage implicite ressources preproc_long_bm25_stem.

Début recherche SÉQUENTIELLE: Index='preprocessed', Modèle='qld', Tag='preproc_long_qld_stem', k=1000
  Fichier de sortie prévu: /content/ap_output/runs/preproc_long_qld_stem.txt
  Initialisation LuceneSearcher...
  LuceneSearcher initialisé.
  Configuration QLD...
  QLD configuré.


Recherche preproc_long_qld_stem:   0%|          | 0/150 [00:00<?, ?it/s]


  150000 lignes résultats écrites dans preproc_long_qld_stem.txt.
Recherche terminée pour preproc_long_qld_stem. Sauvegardé dans /content/ap_output/runs/preproc_long_qld_stem.txt
Temps écoulé: 15.85 secondes.
  Nettoyage implicite ressources preproc_long_qld_stem.

--- Toutes les recherches (Baseline et Stemming) sont terminées. ---

Vérification du contenu de /content/ap_output/runs après les recherches...
total 64784
-rw-r--r-- 1 root root 8040695 Apr  8 23:34 baseline_long_bm25.txt
-rw-r--r-- 1 root root 7876155 Apr  8 23:35 baseline_long_qld.txt
-rw-r--r-- 1 root root 8085341 Apr  8 23:34 baseline_short_bm25.txt
-rw-r--r-- 1 root root 7935476 Apr  8 23:34 baseline_short_qld.txt
-rw-r--r-- 1 root root 8654012 Apr  8 23:35 preproc_long_bm25_stem.txt
-rw-r--r-- 1 root root 8510950 Apr  8 23:36 preproc_long_qld_stem.txt
-rw-r--r-- 1 root root 8686416 Apr  8 23:35 preproc_short_bm25_stem.txt
-rw-r--r-- 1 root root 8537868 Apr  8 23:35 preproc_short_qld_stem.txt


In [ ]:
# === Cellule 5b: Exécuter les Recherches en Parallèle (BM25 & QLD - avec Stemming) ===
# Lance les 8 combinaisons de recherche en parallèle en utilisant multiprocessing.Pool.

from pyserini.search.lucene import LuceneSearcher # Import principal
import time
from tqdm.notebook import tqdm # Pour la barre de progression
import traceback
import os
from jnius import JavaException # Importer seulement JavaException
from multiprocessing import Pool, cpu_count # Importer pour la parallélisation
import math # Pour calculer la taille des chunks

# Définir K_RESULTS
try: K_RESULTS
except NameError: print("Définition K_RESULTS=1000"); K_RESULTS = 1000

# Vérifier variables nécessaires et existence des index
try:
    INDEX_DIR_BASELINE; INDEX_DIR_PREPROC; RUN_DIR; K_RESULTS; CORPUS_DIR;
    queries_short; queries_long; queries_short_preprocessed; queries_long_preprocessed;
    preprocess_text; # Fonction avec stemming doit être définie
    if not os.path.exists(INDEX_DIR_BASELINE): raise FileNotFoundError(f"Index Baseline manquant: {INDEX_DIR_BASELINE}")
    if not os.path.exists(INDEX_DIR_PREPROC): raise FileNotFoundError(f"Index Preprocessed (Stemming) manquant: {INDEX_DIR_PREPROC}")
except NameError as e: print(f"ERREUR: Variable {e} manquante. Exécutez config complète."); raise
except FileNotFoundError as e: print(f"ERREUR: {e}"); raise

# --- Fonction Worker pour la parallélisation ---
def perform_search_single_query_parallel(args):
    """Fonction exécutée par chaque processus pour une seule requête."""
    query_id, query_text, index_path, model, k, run_tag_prefix, use_preprocessed_query, is_preproc_run = args
    # Note: run_tag_prefix est passé, le tag complet est construit ici

    # Construire le tag final à l'intérieur du worker
    run_tag_suffix = "_stem" if is_preproc_run else ""
    final_run_tag = f"{run_tag_prefix}_{model}{run_tag_suffix}" # Ex: baseline_short_bm25 ou preproc_long_qld_stem

    try:
        # Initialiser le searcher DANS le processus fils pour éviter problèmes de sérialisation/JVM
        searcher = LuceneSearcher(index_path)

        # Configurer le modèle de similarité
        if model == 'bm25':
            searcher.set_bm25(k1=0.9, b=0.4)
        elif model == 'qld':
            searcher.set_qld()
        else:
            searcher.set_bm25() # Défaut

        # Prétraiter la requête si nécessaire (uniquement pour les runs baseline ici)
        search_text = preprocess_text(query_text) if use_preprocessed_query else query_text

        # Gérer les requêtes vides après traitement
        if not search_text.strip():
            return [] # Retourner une liste vide si la requête est vide

        # Exécuter la recherche
        hits = searcher.search(search_text, k=k)

        # Formater les résultats pour cette requête
        query_results = []
        for i in range(len(hits)):
            rank = i + 1
            doc_id = hits[i].docid
            score = hits[i].score
            if doc_id is None: continue # Ignorer si docid est None
            # Format TREC: qid Q0 docid rank score run_tag
            query_results.append(f"{query_id} Q0 {doc_id} {rank} {score:.6f} {final_run_tag}\n")

        # Libérer explicitement le searcher (peut aider à gérer les ressources Java)
        del searcher
        return query_results

    except Exception as e:
        # Afficher l'erreur mais ne pas faire planter tout le pool
        print(f"\nERREUR dans worker pour QID {query_id} ({final_run_tag}): {e}")
        # print(traceback.format_exc()) # Décommenter pour trace complète si besoin
        return [] # Retourne une liste vide en cas d'erreur


# --- Fonction Principale pour lancer une recherche parallèle ---
def run_search_parallel_qld(queries, index_path, model, k, run_tag_prefix_arg, use_preprocessed_query=False):
    """Exécute la recherche en parallèle pour un ensemble de requêtes."""
    # Note: Renommé run_tag_prefix en run_tag_prefix_arg pour éviter conflit avec variable externe si nécessaire
    start_time = time.time()
    # Déterminer si c'est un run prétraité pour le tag et nom de fichier
    is_preproc_run = (index_path == INDEX_DIR_PREPROC)
    model_suffix = "_stem" if is_preproc_run else ""
    base_filename = f"{run_tag_prefix_arg}_{model}{model_suffix}.txt"
    output_run_file = os.path.join(RUN_DIR, base_filename)
    # Le tag complet pour affichage et description tqdm
    full_run_tag_display = f"{run_tag_prefix_arg}_{model}{model_suffix}"

    print(f"\nDébut recherche PARALLÈLE: Index='{os.path.basename(index_path)}', Modèle='{model}', Tag='{full_run_tag_display}', k={k}")
    print(f"  Fichier de sortie prévu: {output_run_file}")

    # Préparer les arguments pour chaque tâche
    tasks = []
    for query_id, query_text in queries.items():
        # Passer run_tag_prefix_arg au worker
        tasks.append((query_id, query_text, index_path, model, k, run_tag_prefix_arg, use_preprocessed_query, is_preproc_run))

    # Définir le nombre de workers
    num_workers = 4
    print(f"  Utilisation de {num_workers} processus parallèles...")

    all_results_list = []
    # Utiliser Pool pour la parallélisation
    try:
        # Calculer chunksize pour tqdm
        chunksize = math.ceil(len(tasks) / num_workers / 4)
        if chunksize == 0: chunksize = 1

        with Pool(num_workers) as pool:
           results_iterator = pool.imap_unordered(perform_search_single_query_parallel, tasks, chunksize=chunksize)
           # Envelopper avec tqdm
           for result in tqdm(results_iterator, total=len(tasks), desc=f"Recherche {full_run_tag_display}"):
               if result: all_results_list.extend(result)

        print(f"\n  {len(all_results_list)} lignes de résultats collectées.")

        # Écrire les résultats dans le fichier de run TREC
        if all_results_list:
            os.makedirs(os.path.dirname(output_run_file), exist_ok=True)
            with open(output_run_file, 'w', encoding='utf-8') as f_out:
               f_out.writelines(all_results_list)
            print(f"  Résultats sauvegardés dans {output_run_file}")
        else:
            print("  Avertissement: Aucun résultat collecté pour ce run parallèle.")

    except Exception as e_pool:
        print(f"\nERREUR MAJEURE pendant l'exécution parallèle de {full_run_tag_display}: {e_pool}")
        print(traceback.format_exc())
    finally:
        pass # Pool est fermé par 'with'

    end_time = time.time()
    print(f"Recherche PARALLÈLE terminée pour {full_run_tag_display}.")
    print(f"Temps écoulé: {end_time - start_time:.2f} secondes.")


# --- Exécution des 8 configurations en parallèle ---
# Note: Utiliser run_search_parallel_qld (nom corrigé de la fonction principale)

print("\n--- DÉBUT DES RECHERCHES PARALLÈLES BASELINE (BM25/QLD) ---")
# Les variables run_tag_X ne sont plus utilisées, le nom de fichier est construit dans la fonction
run_search_parallel_qld(queries_short, INDEX_DIR_BASELINE, 'bm25', K_RESULTS, "baseline_short")
run_search_parallel_qld(queries_short, INDEX_DIR_BASELINE, 'qld', K_RESULTS, "baseline_short")
run_search_parallel_qld(queries_long, INDEX_DIR_BASELINE, 'bm25', K_RESULTS, "baseline_long")
run_search_parallel_qld(queries_long, INDEX_DIR_BASELINE, 'qld', K_RESULTS, "baseline_long")

print("\n--- DÉBUT DES RECHERCHES PARALLÈLES PRÉTRAITÉES (STEMMING - BM25/QLD) ---")
# Les noms de fichiers incluront '_stem' automatiquement car INDEX_DIR_PREPROC est utilisé
run_search_parallel_qld(queries_short_preprocessed, INDEX_DIR_PREPROC, 'bm25', K_RESULTS, "preproc_short", use_preprocessed_query=False)
run_search_parallel_qld(queries_short_preprocessed, INDEX_DIR_PREPROC, 'qld', K_RESULTS, "preproc_short", use_preprocessed_query=False)
run_search_parallel_qld(queries_long_preprocessed, INDEX_DIR_PREPROC, 'bm25', K_RESULTS, "preproc_long", use_preprocessed_query=False)
run_search_parallel_qld(queries_long_preprocessed, INDEX_DIR_PREPROC, 'qld', K_RESULTS, "preproc_long", use_preprocessed_query=False)

print("\n--- Toutes les recherches parallèles (Baseline et Stemming) sont terminées. ---")

# Vérifier si des fichiers ont été créés
print(f"\nVérification du contenu de {RUN_DIR} après les recherches parallèles...")
# Utiliser '!' pour exécuter ls
!ls -l {RUN_DIR}




--- DÉBUT DES RECHERCHES PARALLÈLES BASELINE (BM25/QLD) ---

Début recherche PARALLÈLE: Index='baseline', Modèle='bm25', Tag='baseline_short_bm25', k=1000
  Fichier de sortie prévu: /content/ap_output/runs/baseline_short_bm25.txt
  Utilisation de 4 processus parallèles...


Recherche baseline_short_bm25:   0%|          | 0/150 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# === Cellule 5: Exécuter les Recherches en SÉQUENTIEL (BM25 & QLD - avec Stemming) ===
# Lance les 8 combinaisons de recherche séquentiellement. Plus lent en théorie, souvent plus rapide/stable en pratique sur Colab.

from pyserini.search.lucene import LuceneSearcher # Import principal
import time
from tqdm.notebook import tqdm
import traceback
import os
from jnius import JavaException # Importer seulement JavaException

# Définir K_RESULTS
try: K_RESULTS
except NameError: print("Définition K_RESULTS=1000"); K_RESULTS = 1000

# Vérifier variables nécessaires et existence des index
try:
    INDEX_DIR_BASELINE; INDEX_DIR_PREPROC; RUN_DIR; K_RESULTS; CORPUS_DIR;
    queries_short; queries_long; queries_short_preprocessed; queries_long_preprocessed;
    preprocess_text; # Fonction avec stemming doit être définie
    if not os.path.exists(INDEX_DIR_BASELINE): raise FileNotFoundError(f"Index Baseline manquant: {INDEX_DIR_BASELINE}")
    if not os.path.exists(INDEX_DIR_PREPROC): raise FileNotFoundError(f"Index Preprocessed (Stemming) manquant: {INDEX_DIR_PREPROC}")
    if not os.path.exists(os.path.join(CORPUS_DIR, "ap_docs.jsonl")): raise FileNotFoundError("ap_docs.jsonl manquant.")
    if not os.path.exists(os.path.join(CORPUS_DIR, "ap_docs_preprocessed.jsonl")): raise FileNotFoundError("ap_docs_preprocessed.jsonl (stemmed) manquant.")

except NameError as e: print(f"ERREUR: Variable {e} manquante. Exécutez config complète."); raise
except FileNotFoundError as e: print(f"ERREUR: {e}"); raise

def perform_search_sequential_qld(queries, index_path, model, k, output_run_file_base, run_tag_prefix, use_preprocessed_query=False):
    """Exécute la recherche séquentiellement pour un ensemble de requêtes (BM25 ou QLD)."""
    start_time = time.time()
    # Ajouter '_stem' au tag et au nom de fichier pour les runs prétraités
    is_preproc_run = (index_path == INDEX_DIR_PREPROC)
    run_tag_suffix = "_stem" if is_preproc_run else ""
    model_suffix = "_stem" if is_preproc_run else "" # Suffixe pour nom de fichier aussi
    run_tag = f"{run_tag_prefix}_{model}{run_tag_suffix}"
    # Construire le nom de fichier de sortie final
    # Note: output_run_file_base n'est plus utilisé, on utilise run_tag_prefix directement
    base_filename = f"{run_tag_prefix}_{model}{model_suffix}.txt"
    output_run_file = os.path.join(RUN_DIR, base_filename)

    print(f"\nDébut recherche SÉQUENTIELLE: Index='{os.path.basename(index_path)}', Modèle='{model}', Tag='{run_tag}', k={k}")
    print(f"  Fichier de sortie prévu: {output_run_file}")

    all_results_list = []
    searcher = None

    try:
        print(f"  Initialisation LuceneSearcher..."); searcher = LuceneSearcher(index_path); print(f"  LuceneSearcher initialisé.")

        # Configurer similarité
        if model == 'bm25':
            print("  Configuration BM25..."); searcher.set_bm25(k1=0.9, b=0.4); print("  BM25 configuré.")
        elif model == 'qld': # Utiliser Query Likelihood Dirichlet
            print("  Configuration QLD..."); searcher.set_qld(); print("  QLD configuré.")
        else:
            print(f"Modèle '{model}' non reconnu, utilise BM25 par défaut."); searcher.set_bm25()

        # Itérer sur les requêtes
        query_errors = 0
        if 'preprocess_text' not in globals(): raise NameError("preprocess_text non définie.")

        for query_id, query_text in tqdm(queries.items(), desc=f"Recherche {run_tag}"):
            try:
                # Utiliser la fonction preprocess_text (avec stemming) si nécessaire
                search_text = preprocess_text(query_text) if use_preprocessed_query else query_text
                if not search_text.strip(): continue # Ignorer requêtes vides

                hits = searcher.search(search_text, k=k)

                for i in range(len(hits)):
                    rank, doc_id, score = i + 1, hits[i].docid, hits[i].score
                    if doc_id is None: continue
                    all_results_list.append(f"{query_id} Q0 {doc_id} {rank} {score:.6f} {run_tag}\n")
            except Exception as e_query:
                query_errors += 1
                if query_errors < 5: print(f"\nErreur recherche QID {query_id}: {e_query}")
                elif query_errors == 5: print("\nPlusieurs erreurs recherche...")

        # Écrire résultats
        if all_results_list:
             # S'assurer que le dossier RUN_DIR existe avant d'écrire
             os.makedirs(os.path.dirname(output_run_file), exist_ok=True)
             with open(output_run_file, 'w', encoding='utf-8') as f_out: f_out.writelines(all_results_list)
             print(f"\n  {len(all_results_list)} lignes résultats écrites dans {os.path.basename(output_run_file)}.")
        else: print("\n  Avertissement: Aucun résultat généré pour ce run.")
        if query_errors > 0: print(f"  Avertissement: {query_errors} erreurs sur requêtes.")

        end_time = time.time()
        print(f"Recherche terminée pour {run_tag}. Sauvegardé dans {output_run_file}")
        print(f"Temps écoulé: {end_time - start_time:.2f} secondes.")
    except Exception as e_main: print(f"\nERREUR MAJEURE run {run_tag}: {e_main}"); traceback.print_exc()
    finally:
        if searcher: print(f"  Nettoyage implicite ressources {run_tag}.")

# --- Exécution des 8 configurations (BM25 et QLD, avec index stemmatisé pour preproc) ---
print("\n--- DÉBUT DES RECHERCHES BASELINE (BM25/QLD) ---")
# Passer le préfixe du tag directement à la fonction
perform_search_sequential_qld(queries_short, INDEX_DIR_BASELINE, 'bm25', K_RESULTS, "baseline_short", "baseline_short")
perform_search_sequential_qld(queries_short, INDEX_DIR_BASELINE, 'qld', K_RESULTS, "baseline_short", "baseline_short")
perform_search_sequential_qld(queries_long, INDEX_DIR_BASELINE, 'bm25', K_RESULTS, "baseline_long", "baseline_long")
perform_search_sequential_qld(queries_long, INDEX_DIR_BASELINE, 'qld', K_RESULTS, "baseline_long", "baseline_long")
print("\n--- DÉBUT DES RECHERCHES PRÉTRAITÉES (STEMMING - BM25/QLD) ---")
# Les noms de fichiers incluront '_stem' automatiquement car INDEX_DIR_PREPROC est utilisé
perform_search_sequential_qld(queries_short_preprocessed, INDEX_DIR_PREPROC, 'bm25', K_RESULTS, "preproc_short", "preproc_short", use_preprocessed_query=False)
perform_search_sequential_qld(queries_short_preprocessed, INDEX_DIR_PREPROC, 'qld', K_RESULTS, "preproc_short", "preproc_short", use_preprocessed_query=False)
perform_search_sequential_qld(queries_long_preprocessed, INDEX_DIR_PREPROC, 'bm25', K_RESULTS, "preproc_long", "preproc_long", use_preprocessed_query=False)
perform_search_sequential_qld(queries_long_preprocessed, INDEX_DIR_PREPROC, 'qld', K_RESULTS, "preproc_long", "preproc_long", use_preprocessed_query=False)
print("\n--- Toutes les recherches (Baseline et Stemming) sont terminées. ---")

# Vérifier si des fichiers ont été créés
print(f"\nVérification du contenu de {RUN_DIR} après les recherches...")
!ls -l {RUN_DIR}




--- DÉBUT DES RECHERCHES BASELINE (BM25/QLD) ---

Début recherche SÉQUENTIELLE: Index='baseline', Modèle='bm25', Tag='baseline_short_bm25', k=1000
  Fichier de sortie prévu: /content/ap_output/runs/baseline_short_bm25.txt
  Initialisation LuceneSearcher...
  LuceneSearcher initialisé.
  Configuration BM25...
  BM25 configuré.


Recherche baseline_short_bm25:   0%|          | 0/150 [00:00<?, ?it/s]


  150000 lignes résultats écrites dans baseline_short_bm25.txt.
Recherche terminée pour baseline_short_bm25. Sauvegardé dans /content/ap_output/runs/baseline_short_bm25.txt
Temps écoulé: 14.77 secondes.
  Nettoyage implicite ressources baseline_short_bm25.

Début recherche SÉQUENTIELLE: Index='baseline', Modèle='qld', Tag='baseline_short_qld', k=1000
  Fichier de sortie prévu: /content/ap_output/runs/baseline_short_qld.txt
  Initialisation LuceneSearcher...
  LuceneSearcher initialisé.
  Configuration QLD...
  QLD configuré.


Recherche baseline_short_qld:   0%|          | 0/150 [00:00<?, ?it/s]


  150000 lignes résultats écrites dans baseline_short_qld.txt.
Recherche terminée pour baseline_short_qld. Sauvegardé dans /content/ap_output/runs/baseline_short_qld.txt
Temps écoulé: 16.05 secondes.
  Nettoyage implicite ressources baseline_short_qld.

Début recherche SÉQUENTIELLE: Index='baseline', Modèle='bm25', Tag='baseline_long_bm25', k=1000
  Fichier de sortie prévu: /content/ap_output/runs/baseline_long_bm25.txt
  Initialisation LuceneSearcher...
  LuceneSearcher initialisé.
  Configuration BM25...
  BM25 configuré.


...


  150000 lignes résultats écrites dans preproc_short_qld_stem.txt.
Recherche terminée pour preproc_short_qld_stem. Sauvegardé dans /content/ap_output/runs/preproc_short_qld_stem.txt
Temps écoulé: 13.28 secondes.
  Nettoyage implicite ressources preproc_short_qld_stem.

Début recherche SÉQUENTIELLE: Index='preprocessed', Modèle='bm25', Tag='preproc_long_bm25_stem', k=1000
  Fichier de sortie prévu: /content/ap_output/runs/preproc_long_bm25_stem.txt
  Initialisation LuceneSearcher...
  LuceneSearcher initialisé.
  Configuration BM25...
  BM25 configuré.


Recherche preproc_long_bm25_stem:   0%|          | 0/150 [00:00<?, ?it/s]


  150000 lignes résultats écrites dans preproc_long_bm25_stem.txt.
Recherche terminée pour preproc_long_bm25_stem. Sauvegardé dans /content/ap_output/runs/preproc_long_bm25_stem.txt
Temps écoulé: 14.51 secondes.
  Nettoyage implicite ressources preproc_long_bm25_stem.

Début recherche SÉQUENTIELLE: Index='preprocessed', Modèle='qld', Tag='preproc_long_qld_stem', k=1000
  Fichier de sortie prévu: /content/ap_output/runs/preproc_long_qld_stem.txt
  Initialisation LuceneSearcher...
  LuceneSearcher initialisé.
  Configuration QLD...
  QLD configuré.


Recherche preproc_long_qld_stem:   0%|          | 0/150 [00:00<?, ?it/s]


  150000 lignes résultats écrites dans preproc_long_qld_stem.txt.
Recherche terminée pour preproc_long_qld_stem. Sauvegardé dans /content/ap_output/runs/preproc_long_qld_stem.txt
Temps écoulé: 15.29 secondes.
  Nettoyage implicite ressources preproc_long_qld_stem.

--- Toutes les recherches (Baseline et Stemming) sont terminées. ---

Vérification du contenu de /content/ap_output/runs après les recherches...
total 64784
-rw-r--r-- 1 root root 8040695 Apr  9 00:29 baseline_long_bm25.txt
-rw-r--r-- 1 root root 7876155 Apr  9 00:29 baseline_long_qld.txt
-rw-r--r-- 1 root root 8085341 Apr  9 00:28 baseline_short_bm25.txt
-rw-r--r-- 1 root root 7935476 Apr  9 00:29 baseline_short_qld.txt
-rw-r--r-- 1 root root 8654012 Apr  9 00:30 preproc_long_bm25_stem.txt
-rw-r--r-- 1 root root 8510950 Apr  9 00:30 preproc_long_qld_stem.txt
-rw-r--r-- 1 root root 8686416 Apr  9 00:29 preproc_short_bm25_stem.txt
-rw-r--r-- 1 root root 8537868 Apr  9 00:30 preproc_short_qld_stem.txt


In [ ]:
# === Cellule 6: Évaluation des Runs (Stemming vs Baseline) ===
# Lit les fichiers Qrels, lit les fichiers de résultats (.txt) du dossier RUN_DIR,
# calcule MAP et P@10, et affiche/sauvegarde les tableaux récapitulatifs.
# Devrait maintenant évaluer les runs baseline (BM25/QLD) et preproc_stem (BM25/QLD).

import pandas as pd
import glob
import pytrec_eval
import os
import traceback

# Vérifier que les chemins sont définis
try:
    QRELS_DIR
    RUN_DIR
    EVAL_DIR
except NameError:
    print("ERREUR: Variables de chemin non définies. Exécutez la cellule de configuration complète.")
    raise

print(f"Préparation Qrels depuis: {QRELS_DIR}")
qrels_files = sorted(glob.glob(os.path.join(QRELS_DIR, "qrels.*.txt")))
if not qrels_files: print(f"ATTENTION: Aucun fichier Qrels trouvé dans {QRELS_DIR}."); qrels_dict = {}
else:
    print(f"Fichiers Qrels trouvés: {qrels_files}")
    all_qrels_data = []
    for qf in qrels_files:
        try:
            # Lire le fichier qrels en spécifiant les types pour éviter les erreurs
            qrels_df = pd.read_csv(qf, sep='\s+', names=['query_id', 'unused', 'doc_id', 'relevance'],
                                   dtype={'query_id': str, 'unused': str, 'doc_id': str, 'relevance': int})
            all_qrels_data.append(qrels_df[['query_id', 'doc_id', 'relevance']])
        except Exception as e: print(f"Erreur lecture Qrels {qf}: {e}")
    if not all_qrels_data: print("ERREUR: Impossible lire données Qrels."); qrels_dict = {}
    else:
        combined_qrels_df = pd.concat(all_qrels_data, ignore_index=True)
        qrels_dict = {}
        # Convertir le DataFrame en dictionnaire attendu par pytrec_eval
        for _, row in combined_qrels_df.iterrows():
            qid, did, rel = str(row['query_id']), str(row['doc_id']), int(row['relevance'])
            if rel < 0: continue # Ignorer jugements négatifs
            if qid not in qrels_dict: qrels_dict[qid] = {}
            qrels_dict[qid][did] = rel
        print(f"Total {len(qrels_dict)} requêtes avec jugements chargées.")

# --- Évaluation des Runs ---
if not qrels_dict: print("\nAucun jugement de pertinence chargé, impossible d'évaluer.")
else:
    measures = {'map', 'P_10'} # Métriques à calculer
    evaluator = pytrec_eval.RelevanceEvaluator(qrels_dict, measures) # Initialiser l'évaluateur
    # Trouver tous les fichiers .txt dans le dossier des runs
    run_files = sorted(glob.glob(os.path.join(RUN_DIR, "*.txt")))
    print(f"\n{len(run_files)} fichiers de run à évaluer trouvés dans {RUN_DIR}.")
    print(f"  Fichiers: {[os.path.basename(f) for f in run_files]}") # Afficher les noms

    results_summary = [] # Liste pour stocker les résultats agrégés
    if not run_files: print(f"ATTENTION: Aucun fichier de run (.txt) trouvé dans {RUN_DIR}.")
    else:
        # Boucler sur chaque fichier de run trouvé
        for run_file in run_files:
            run_name = os.path.basename(run_file)
            # Ignorer les anciens runs RM3 s'ils existent encore (basés sur lemmatisation)
            # Ajustement: Ignorer tout run RM3 qui ne se termine PAS par _rm3_stem.txt
            # Cela permet d'ignorer un éventuel preproc_long_bm25_rm3.txt de l'essai précédent.
            if 'rm3' in run_name and not run_name.endswith('_rm3_stem.txt'):
                 print(f"\n--- Ignorer ancien run RM3 (non-stem): {run_name} ---")
                 continue

            print(f"\n--- Évaluation: {run_name} ---")
            run_dict = {} # Dictionnaire pour stocker les résultats de ce run
            error_count = 0
            line_count = 0
            try:
                # Lire le fichier run ligne par ligne
                with open(run_file, 'r', encoding='utf-8') as f_run:
                    for line in f_run:
                        line_count += 1
                        parts = line.strip().split()
                        # Vérifier le format TREC (6 colonnes)
                        if len(parts) != 6: error_count += 1; continue
                        qid, _, did, _, score, _ = parts # Extraire les infos utiles
                        try: score = float(score) # Convertir le score en float
                        except ValueError: error_count += 1; continue
                        qid = str(qid) # Assurer que qid est une chaîne
                        # Stocker le score pour ce document et cette requête
                        if qid not in run_dict: run_dict[qid] = {}
                        run_dict[qid][did] = score
                if error_count > 0: print(f"  Avertissement: {error_count} lignes mal formatées ignorées sur {line_count} lignes.")

                # Filtrer le run pour ne garder que les requêtes présentes dans les Qrels
                filtered_run_dict = {qid: docs for qid, docs in run_dict.items() if qid in qrels_dict}
                ignored_q = len(run_dict) - len(filtered_run_dict)
                if ignored_q > 0: print(f"  Avertissement: {ignored_q} requêtes run ignorées (absentes Qrels).") # Normal, basé sur 51 qrels
                if not filtered_run_dict: print("  Erreur: Aucune requête ne correspond aux Qrels."); continue

                # Évaluer le run filtré avec pytrec_eval
                eval_results = evaluator.evaluate(filtered_run_dict)
                # Calculer les moyennes des métriques sur toutes les requêtes évaluées
                all_maps = [q_res.get("map", 0) for q_res in eval_results.values()]
                all_p10s = [q_res.get("P_10", 0) for q_res in eval_results.values()]
                avg_map = sum(all_maps) / len(all_maps) if all_maps else 0
                avg_p10 = sum(all_p10s) / len(all_p10s) if all_p10s else 0

                # Afficher les résultats moyens pour ce run
                print(f"  MAP: {avg_map:.4f}")
                print(f"  P@10: {avg_p10:.4f}")

                # Extraire les informations du nom de fichier pour le résumé
                run_name_parts = run_name.replace('.txt','')
                parts = run_name_parts.split('_')
                if len(parts) >= 3:
                    index_type = parts[0] # baseline ou preproc
                    query_type = parts[1] # short ou long
                    # Gérer les suffixes _stem et _rm3
                    model_parts = parts[2:]
                    model_suffix = ""
                    # Vérifier si le dernier élément est 'rm3'
                    if model_parts[-1] == 'rm3':
                        model_suffix += "+RM3"
                        model_parts = model_parts[:-1] # Enlever 'rm3'
                    # Vérifier si le dernier élément (restant) est 'stem'
                    if model_parts and model_parts[-1] == 'stem':
                        # On note que c'est stem via index_type='preproc', on enlève 'stem' du nom modèle
                        model_parts = model_parts[:-1]

                    model_type = "_".join(model_parts) # Devrait être BM25 ou QLD

                    # Utiliser 'Preprocessed (Stem)' comme nom d'index pour plus de clarté
                    display_index_type = "Preprocessed (Stem)" if index_type == "preproc" else "Baseline"

                    results_summary.append({
                        "Run Name": run_name, "Index": display_index_type, # Utiliser le nom plus clair
                        "Query Type": query_type.capitalize(),
                        "Weighting Scheme": model_type.upper() + model_suffix, # Ex: BM25, QLD, BM25+RM3
                        "MAP": avg_map, "P@10": avg_p10
                    })
                else: print(f"  Avertissement: Impossible parser nom run '{run_name}'.")

            except FileNotFoundError: print(f"  Erreur: Fichier run non trouvé: {run_file}")
            except Exception as e: print(f"  Erreur évaluation {run_name}: {e}"); traceback.print_exc()

        # Afficher et sauvegarder le résumé final
        if results_summary:
            print("\n\n=== Tableau Récapitulatif (Stemming vs Baseline) ===")
            results_df = pd.DataFrame(results_summary)
            # Trier pour une meilleure lisibilité
            results_df = results_df.sort_values(by=["Index", "Query Type", "Weighting Scheme"])

            # Afficher le DataFrame complet
            print("\n--- Résultats Complets ---")
            print(results_df.to_markdown(index=False, floatfmt=".4f"))

            # Essayer d'afficher les tableaux pivots
            try:
                pivot_map = results_df.pivot_table(index=['Query Type', 'Weighting Scheme'], columns='Index', values='MAP')
                print("\n--- MAP (Tableau Pivot) ---")
                print(pivot_map.to_markdown(floatfmt=".4f"))
            except Exception as e_pivot: print(f"\n(Erreur création tableau pivot MAP: {e_pivot})")

            try:
                pivot_p10 = results_df.pivot_table(index=['Query Type', 'Weighting Scheme'], columns='Index', values='P@10')
                print("\n--- P@10 (Tableau Pivot) ---")
                print(pivot_p10.to_markdown(floatfmt=".4f"))
            except Exception as e_pivot: print(f"\n(Erreur création tableau pivot P@10: {e_pivot})")

            # Sauvegarder le DataFrame complet final avec un nom spécifique pour le stemming
            summary_file_path = os.path.join(EVAL_DIR, "evaluation_summary_stemming_final.csv")
            try:
                 results_df.to_csv(summary_file_path, index=False)
                 print(f"\nTableau récapitulatif complet sauvegardé: {summary_file_path}")
            except Exception as e_save: print(f"\nErreur sauvegarde résumé: {e_save}")
        else: print("\nAucun résultat d'évaluation à afficher.")



Préparation Qrels depuis: /content/drive/My Drive/Projet_RI/TREC/jugements de pertinence
Fichiers Qrels trouvés: ['/content/drive/My Drive/Projet_RI/TREC/jugements de pertinence/qrels.1-50.AP8890.txt', '/content/drive/My Drive/Projet_RI/TREC/jugements de pertinence/qrels.101-150.AP8890.txt', '/content/drive/My Drive/Projet_RI/TREC/jugements de pertinence/qrels.51-100.AP8890.txt']
Total 150 requêtes avec jugements chargées.

8 fichiers de run à évaluer trouvés dans /content/ap_output/runs.
  Fichiers: ['baseline_long_bm25.txt', 'baseline_long_qld.txt', 'baseline_short_bm25.txt', 'baseline_short_qld.txt', 'preproc_long_bm25_stem.txt', 'preproc_long_qld_stem.txt', 'preproc_short_bm25_stem.txt', 'preproc_short_qld_stem.txt']

--- Évaluation: baseline_long_bm25.txt ---
  Avertissement: 99 requêtes run ignorées (absentes Qrels).
  MAP: 0.2205
  P@10: 0.4765

--- Évaluation: baseline_long_qld.txt ---
  Avertissement: 99 requêtes run ignorées (absentes Qrels).
  MAP: 0.2171
  P@10: 0.4804

---

In [ ]:
# === Cellule de Vérification des Dossiers d'Index ===
# Utilise les commandes shell de Colab préfixées par '!'

import os # Importer os pour définir les variables si besoin

# Définir les chemins au cas où ils ne seraient pas dans l'environnement
# (Normalement définis par la cellule de configuration complète)
OUTPUT_DIR = "/content/ap_output"
INDEX_DIR_BASELINE = os.path.join(OUTPUT_DIR, "indexes/baseline")
INDEX_DIR_PREPROC = os.path.join(OUTPUT_DIR, "indexes/preprocessed")

print("--- Contenu Index Baseline ---")
# Utiliser '!' pour exécuter ls -lh sur le dossier baseline
# Mettre le chemin entre guillemets pour gérer les espaces potentiels
!ls -lh "{INDEX_DIR_BASELINE}"

print("\n--- Contenu Index Preprocessed (Stemming) ---")
# Utiliser '!' pour exécuter ls -lh sur le dossier preprocessed
!ls -lh "{INDEX_DIR_PREPROC}"

# Ajouter une vérification d'existence pour être plus clair
print("\n--- Vérification d'existence ---")
if os.path.exists(INDEX_DIR_BASELINE):
    print(f"Le dossier {INDEX_DIR_BASELINE} existe.")
else:
    print(f"ATTENTION: Le dossier {INDEX_DIR_BASELINE} N'EXISTE PAS.")

if os.path.exists(INDEX_DIR_PREPROC):
    print(f"Le dossier {INDEX_DIR_PREPROC} existe.")
else:
    print(f"ATTENTION: Le dossier {INDEX_DIR_PREPROC} N'EXISTE PAS.")



--- Contenu Index Baseline ---
total 0

--- Contenu Index Preprocessed (Stemming) ---
total 0

--- Vérification d'existence ---
Le dossier /content/ap_output/indexes/baseline existe.
Le dossier /content/ap_output/indexes/preprocessed existe.


In [ ]:
!ls -ld /content/ap_output


drwxr-xr-x 6 root root 4096 Apr  8 22:43 /content/ap_output


In [ ]:
# === Cellule 1: Extraire, Décompresser et Formater les Documents ===
# Lit AP.tar, décompresse les .gz internes, extrait <DOC>, <DOCNO>, <TEXT>
# et écrit le résultat dans ap_docs.jsonl.

import tarfile
import re
import json
import gzip # Importer le module gzip
from tqdm.notebook import tqdm
import os
import traceback

# Vérifier que les chemins sont définis (normalement fait par la cellule de config)
try:
    AP_TAR_PATH
    CORPUS_DIR
except NameError:
    print("ERREUR: Variables de chemin non définies. Exécutez la cellule de configuration complète.")
    raise

JSONL_OUTPUT_PATH = os.path.join(CORPUS_DIR, "ap_docs.jsonl")

print(f"Extraction, Décompression et Formatage depuis {AP_TAR_PATH} vers {JSONL_OUTPUT_PATH}...")

# Vérifier si le fichier AP.tar existe
if not os.path.exists(AP_TAR_PATH):
    raise FileNotFoundError(f"Le fichier d'archive {AP_TAR_PATH} n'a pas été trouvé.")
else:
    tar_size = os.path.getsize(AP_TAR_PATH)
    print(f"  Taille du fichier {AP_TAR_PATH}: {tar_size} octets.") # Devrait être ~275Mo

# Regex pour extraire les infos
doc_pattern = re.compile(r"<DOC>(.*?)</DOC>", re.DOTALL)
docno_pattern = re.compile(r"<DOCNO>\s*(.*?)\s*</DOCNO>")
text_pattern = re.compile(r"<TEXT>(.*?)</TEXT>", re.DOTALL)

# Compteurs
doc_count = 0
file_read_count = 0
skipped_members = 0
decompression_errors = 0

try:
    # Ouvrir le fichier de sortie et l'archive TAR
    with open(JSONL_OUTPUT_PATH, 'w', encoding='utf-8') as outfile, tarfile.open(AP_TAR_PATH, "r:") as tar:
        members = tar.getmembers()
        print(f"\n{len(members)} membres trouvés dans l'archive TAR.") # Devrait être ~1051

        # Boucler sur chaque membre de l'archive
        for member in tqdm(members, desc="Traitement des fichiers TAR"):
            # Ignorer si ce n'est pas un fichier .gz ou .Z
            if not member.isfile() or not member.name.lower().endswith(('.gz', '.z')):
                skipped_members += 1
                continue

            file_read_count += 1
            content = "" # Réinitialiser pour chaque fichier

            try:
                # Extraire le contenu compressé
                f = tar.extractfile(member)
                if f:
                    compressed_content = f.read()
                    f.close()

                    # Décompresser le contenu
                    try:
                        content_bytes = gzip.decompress(compressed_content)
                        content = content_bytes.decode('utf-8', errors='ignore') # Décoder après décompression
                    except gzip.BadGzipFile: # Gérer si ce n'est pas du gzip
                        content = compressed_content.decode('utf-8', errors='ignore')
                        decompression_errors += 1
                    except Exception as e_gzip:
                         print(f"\nErreur de décompression pour {member.name}: {e_gzip}")
                         decompression_errors += 1
                         continue # Passer au suivant

                    # Trouver tous les blocs <DOC> dans le contenu décompressé
                    doc_matches = doc_pattern.findall(content)
                    if not doc_matches: continue # Passer si aucun doc trouvé

                    # Boucler sur chaque document trouvé
                    for doc_content in doc_matches:
                        docno_match = docno_pattern.search(doc_content)
                        if not docno_match: continue
                        doc_id = docno_match.group(1).strip()

                        text_match = text_pattern.search(doc_content)
                        # Nettoyer le texte (espaces multiples)
                        doc_text = ' '.join(text_match.group(1).strip().split()) if text_match else ""

                        # Écrire la ligne JSONL
                        try:
                            json_line = json.dumps({"id": str(doc_id), "contents": str(doc_text)})
                            outfile.write(json_line + '\n')
                            doc_count += 1
                        except Exception as e_write:
                            print(f"Erreur écriture JSON pour doc_id {doc_id}: {e_write}")

            except KeyError as e_key: print(f"\nAvertissement: Membre '{member.name}' inaccessible (KeyError): {e_key}"); skipped_members += 1
            except EOFError: print(f"\nAvertissement: Fin fichier inattendue {member.name}."); skipped_members += 1
            except Exception as e_extract: print(f"\nErreur extraction/lecture {member.name}: {e_extract}"); skipped_members += 1

except tarfile.ReadError as e_tar: print(f"\nERREUR lecture TAR {AP_TAR_PATH}: {e_tar}"); raise e_tar
except FileNotFoundError: print(f"\nERREUR: Fichier TAR {AP_TAR_PATH} non trouvé."); raise FileNotFoundError
except Exception as e_general: print(f"\nERREUR générale traitement TAR: {e_general}"); traceback.print_exc(); raise e_general

# Afficher le résumé de l'extraction
print(f"\n--- Fin de l'Extraction et Décompression ---")
print(f"  {file_read_count} fichiers (.gz/.Z) lus.")
print(f"  {skipped_members} membres ignorés.")
if decompression_errors > 0: print(f"  {decompression_errors} erreurs/avertissements décompression.")
print(f"  {doc_count} documents écrits dans {JSONL_OUTPUT_PATH}") # Devrait être ~240k

# Vérifier la taille du fichier de sortie
if os.path.exists(JSONL_OUTPUT_PATH):
    output_size = os.path.getsize(JSONL_OUTPUT_PATH)
    print(f"  Taille finale {JSONL_OUTPUT_PATH}: {output_size} octets.") # Devrait être ~600Mo
    if output_size > 0 and doc_count > 0: print("  SUCCÈS: Fichier de sortie contient des données.")
    else: print("  ATTENTION: Fichier de sortie vide ou aucun document écrit.")
else: print(f"  ATTENTION: Fichier {JSONL_OUTPUT_PATH} non créé.")



Extraction, Décompression et Formatage depuis /content/drive/My Drive/Projet_RI/TREC/AP.tar vers /content/ap_output/corpus/ap_docs.jsonl...
  Taille du fichier /content/drive/My Drive/Projet_RI/TREC/AP.tar: 288296960 octets.

1051 membres trouvés dans l'archive TAR.


Traitement des fichiers TAR:   0%|          | 0/1051 [00:00<?, ?it/s]


--- Fin de l'Extraction et Décompression ---
  1050 fichiers (.gz/.Z) lus.
  1 membres ignorés.
  242918 documents écrits dans /content/ap_output/corpus/ap_docs.jsonl
  Taille finale /content/ap_output/corpus/ap_docs.jsonl: 620216647 octets.
  SUCCÈS: Fichier de sortie contient des données.


In [ ]:
# === Sauvegarde du dossier /content/ap_output vers Google Drive ===
import os
import subprocess
import time

# Chemin de base du projet sur Drive (vérifiez qu'il est correct)
# Devrait être défini par la cellule de configuration complète
try:
    DRIVE_PROJECT_PATH
except NameError:
    print("ERREUR: La variable DRIVE_PROJECT_PATH n'est pas définie. Exécutez d'abord la cellule de configuration complète.")
    # Optionnel: Redéfinir ici si nécessaire, mais il vaut mieux exécuter la cellule de setup
    # DRIVE_PROJECT_PATH = "/content/drive/My Drive/Projet_RI/TREC"
    raise

# Dossier source dans Colab à sauvegarder
SOURCE_DIR_TO_SAVE = "/content/ap_output"

# Dossier cible sur Google Drive pour la sauvegarde
# (Crée un sous-dossier 'colab_output_backup' dans votre dossier projet TREC)
TARGET_BACKUP_DIR_ON_DRIVE = os.path.join(DRIVE_PROJECT_PATH, "colab_output_backup")

print(f"Source à sauvegarder : {SOURCE_DIR_TO_SAVE}")
print(f"Cible sur Drive : {TARGET_BACKUP_DIR_ON_DRIVE}")

# Vérifier si le dossier source existe
if os.path.exists(SOURCE_DIR_TO_SAVE):
    # Créer le dossier cible sur Drive s'il n'existe pas
    print(f"Création (si nécessaire) du dossier cible: {TARGET_BACKUP_DIR_ON_DRIVE}")
    os.makedirs(TARGET_BACKUP_DIR_ON_DRIVE, exist_ok=True)

    print("\nCopie des fichiers vers Google Drive en cours... (Cela peut prendre plusieurs minutes)")
    # Utiliser cp -r (récursif), -u (update: copie seulement si plus récent ou manquant), -v (verbeux)
    # Copie le contenu de SOURCE_DIR_TO_SAVE dans TARGET_BACKUP_DIR_ON_DRIVE
    # L'option -u évite de recopier inutilement les gros index s'ils n'ont pas changé,
    # mais écrase les fichiers runs/ et eval/ s'ils existaient déjà avec le même nom.
    copy_cmd = f"cp -r -u -v '{SOURCE_DIR_TO_SAVE}/.' '{TARGET_BACKUP_DIR_ON_DRIVE}/'"
    try:
        # Exécuter la commande de copie
        process = subprocess.run(copy_cmd, shell=True, check=True, capture_output=True, text=True, timeout=900) # Timeout 15 minutes
        # Afficher la fin de la sortie pour confirmation
        print("... (Sortie de la copie, peut être longue) ...")
        # stderr affiche souvent les fichiers copiés avec -v
        stderr_lines = process.stderr.splitlines()
        print("\nExtrait de la fin de la sortie STDERR (fichiers copiés):")
        for line in stderr_lines[-20:]: # Afficher les 20 dernières lignes
             print(line)

        print("\nSauvegarde terminée avec succès !")
        print(f"Le contenu de {SOURCE_DIR_TO_SAVE} a été copié/mis à jour dans {TARGET_BACKUP_DIR_ON_DRIVE}")
        # Vérifier rapidement si les dossiers runs et eval existent dans la sauvegarde
        print("\nVérification de la sauvegarde sur Drive (partiel):")
        print(f"Contenu de {TARGET_BACKUP_DIR_ON_DRIVE}/runs :")
        !ls -l "{TARGET_BACKUP_DIR_ON_DRIVE}/runs" | head -n 10
        print(f"\nContenu de {TARGET_BACKUP_DIR_ON_DRIVE}/eval :")
        !ls -l "{TARGET_BACKUP_DIR_ON_DRIVE}/eval"
    except subprocess.CalledProcessError as e:
         print(f"\nERREUR lors de la sauvegarde (code {e.returncode}).")
         print("STDOUT:", e.stdout)
         print("STDERR:", e.stderr)
         print("\nVérifiez que vous avez les permissions d'écriture sur Google Drive.")
    except subprocess.TimeoutExpired as e:
        print(f"\nERREUR: La sauvegarde a dépassé le délai d'attente.")
    except Exception as e:
        print(f"\nERREUR inattendue lors de la sauvegarde: {e}")
else:
    print(f"Le dossier source {SOURCE_DIR_TO_SAVE} n'existe pas, aucune sauvegarde effectuée. Avez-vous exécuté la configuration complète ?")




Source à sauvegarder : /content/ap_output
Cible sur Drive : /content/drive/My Drive/Projet_RI/TREC/colab_output_backup
Création (si nécessaire) du dossier cible: /content/drive/My Drive/Projet_RI/TREC/colab_output_backup

Copie des fichiers vers Google Drive en cours... (Cela peut prendre plusieurs minutes)
... (Sortie de la copie, peut être longue) ...

Extrait de la fin de la sortie STDERR (fichiers copiés):

Sauvegarde terminée avec succès !
Le contenu de /content/ap_output a été copié/mis à jour dans /content/drive/My Drive/Projet_RI/TREC/colab_output_backup

Vérification de la sauvegarde sur Drive (partiel):
Contenu de /content/drive/My Drive/Projet_RI/TREC/colab_output_backup/runs :
total 64774
-rw------- 1 root root 8040695 Apr  9 01:12 baseline_long_bm25.txt
-rw------- 1 root root 7876155 Apr  9 01:12 baseline_long_qld.txt
-rw------- 1 root root 8085341 Apr  9 01:12 baseline_short_bm25.txt
-rw------- 1 root root 7935476 Apr  9 01:12 baseline_short_qld.txt
-rw------- 1 root root

In [ ]:
# === Cellule 7: Exécuter la Recherche Améliorée (RM3) ===
# Applique RM3 sur la meilleure configuration de base identifiée à l'étape 6 (Stemming vs Baseline).
# !! N'OUBLIEZ PAS DE CONFIGURER LES VARIABLES BEST_... CI-DESSOUS !!

from pyserini.search.lucene import LuceneSearcher
from jnius import autoclass, JavaException
from tqdm.notebook import tqdm
import time
import traceback
import os
import re # Importer re pour l'analyse de sortie éventuelle

# Vérifier variables nécessaires
try: INDEX_DIR_BASELINE; INDEX_DIR_PREPROC; RUN_DIR; K_RESULTS; EVAL_DIR; queries_short; queries_long; queries_short_preprocessed; queries_long_preprocessed; preprocess_text;
except NameError as e: print(f"ERREUR: Variable {e} manquante."); raise

# --- À CONFIGURER selon vos meilleurs résultats de l'Étape 6 (Stemming vs Baseline) ---
print("--- Configuration RM3 ---")
print("Veuillez éditer les variables BEST_... ci-dessous en fonction de vos meilleurs résultats MAP de l'étape précédente (évaluation avec stemming).")
# Exemple: si baseline_long_bm25 était le meilleur (MAP ~0.2205)
BEST_INDEX_PATH = INDEX_DIR_BASELINE           # Mettez INDEX_DIR_BASELINE ou INDEX_DIR_PREPROC
BEST_QUERIES = queries_long                  # Mettez queries_short, queries_long, queries_short_preprocessed, ou queries_long_preprocessed
BEST_MODEL_BASE = 'bm25'                      # Mettez 'bm25' ou 'qld'
BEST_RUN_TAG_PREFIX = "baseline_long"          # Mettez 'baseline_short', 'baseline_long', 'preproc_short', ou 'preproc_long'
USE_PREPROC_QUERY_FOR_RM3 = False             # Mettez False si index baseline ou si BEST_QUERIES est _preprocessed, True si index preproc ET BEST_QUERIES est brut (peu courant)
# ----------------------------------------------------------------
print(f"Configuration choisie pour RM3:")
print(f"  Index: {os.path.basename(BEST_INDEX_PATH)}")
print(f"  Modèle Base: {BEST_MODEL_BASE}")
print(f"  Préfixe Tag: {BEST_RUN_TAG_PREFIX}")

# Nom du fichier et tag pour le run RM3
# Ajouter '_stem' au tag et nom de fichier si basé sur l'index preproc (stemming)
run_tag_suffix_rm3 = "_stem" if (BEST_INDEX_PATH == INDEX_DIR_PREPROC) else ""
PRF_RUN_FILE = os.path.join(RUN_DIR, f"{BEST_RUN_TAG_PREFIX}_{BEST_MODEL_BASE}_rm3{run_tag_suffix_rm3}.txt")
RM3_RUN_TAG = f"{BEST_RUN_TAG_PREFIX}_{BEST_MODEL_BASE}_rm3{run_tag_suffix_rm3}"
print(f"  Run Tag: {RM3_RUN_TAG}")
print(f"  Fichier de sortie prévu: {PRF_RUN_FILE}")


# Paramètres RM3 (standards)
rm3_config = {'fb_terms': 10, 'fb_docs': 10, 'original_query_weight': 0.5}
print(f"  Paramètres RM3: {rm3_config}")

# --- Fonction de recherche RM3 (séquentielle) ---
def perform_search_sequential_rm3(queries, index_path, model_base, k, output_run_file, run_tag, use_preprocessed_query=False, rm3_params=None):
    """Exécute la recherche RM3 séquentiellement."""
    start_time = time.time(); print(f"\nDébut recherche RM3: Modèle='{model_base}+RM3', Tag='{run_tag}', k={k}")
    all_results_list = []; searcher = None
    try:
        print(f"  Init LuceneSearcher..."); searcher = LuceneSearcher(index_path); print(f"  Init OK.")
        # Configurer similarité base
        if model_base == 'bm25': print("  Config BM25 (base)..."); searcher.set_bm25(k1=0.9, b=0.4)
        elif model_base == 'qld': print("  Config QLD (base)..."); searcher.set_qld()
        else: print(f"Modèle base '{model_base}' non reconnu, utilise BM25."); searcher.set_bm25()
        # Activer RM3
        print("  Activation RM3..."); searcher.set_rm3(**rm3_params); print("  RM3 activé.")
        # Itérer sur requêtes
        query_errors = 0
        if 'preprocess_text' not in globals(): raise NameError("preprocess_text non définie.")
        for query_id, query_text in tqdm(queries.items(), desc=f"Recherche {run_tag}"):
            try:
                # Prétraitement de la requête seulement si nécessaire
                search_text = preprocess_text(query_text) if use_preprocessed_query else query_text
                if not search_text.strip(): continue
                hits = searcher.search(search_text, k=k)
                for i in range(len(hits)):
                    rank, doc_id, score = i + 1, hits[i].docid, hits[i].score
                    if doc_id is None: continue
                    all_results_list.append(f"{query_id} Q0 {doc_id} {rank} {score:.6f} {run_tag}\n")
            except Exception as e_query: query_errors += 1; # Limiter affichage

        # --- CORRECTION SYNTAXE ICI ---
        # Écrire résultats (avec indentation correcte)
        if all_results_list:
            # S'assurer que le dossier RUN_DIR existe avant d'écrire
            os.makedirs(os.path.dirname(output_run_file), exist_ok=True)
            # Ouvrir le fichier et écrire les lignes
            with open(output_run_file, 'w', encoding='utf-8') as f_out:
                f_out.writelines(all_results_list)
            # Afficher confirmation
            print(f"\n  {len(all_results_list)} lignes résultats écrites: {os.path.basename(output_run_file)}.")
        else:
            print("\n  Avertissement: Aucun résultat RM3 généré.")
        # --- FIN CORRECTION ---

        if query_errors > 0: print(f"  Avertissement: {query_errors} erreurs requêtes.")
        end_time = time.time(); print(f"Recherche RM3 terminée {run_tag}. Temps: {end_time - start_time:.2f}s.")

    except Exception as e_main: print(f"\nERREUR MAJEURE run RM3 {run_tag}: {e_main}"); traceback.print_exc()
    finally:
        if searcher: print(f"  Nettoyage implicite {run_tag}.")

# Lancer la recherche RM3
print("\nLancement de la recherche RM3...")
perform_search_sequential_rm3( BEST_QUERIES, BEST_INDEX_PATH, BEST_MODEL_BASE, K_RESULTS, PRF_RUN_FILE, RM3_RUN_TAG, use_preprocessed_query=USE_PREPROC_QUERY_FOR_RM3, rm3_params=rm3_config)

print("\n--- Exécution recherche RM3 terminée. ---")
# Vérifier création fichier
print(f"\nVérification création fichier {PRF_RUN_FILE}...")
# Utiliser !ls avec des guillemets pour gérer les chemins
!ls -l "{PRF_RUN_FILE}"


--- Configuration RM3 ---
Veuillez éditer les variables BEST_... ci-dessous en fonction de vos meilleurs résultats MAP de l'étape précédente (évaluation avec stemming).
Configuration choisie pour RM3:
  Index: baseline
  Modèle Base: bm25
  Préfixe Tag: baseline_long
  Run Tag: baseline_long_bm25_rm3
  Fichier de sortie prévu: /content/ap_output/runs/baseline_long_bm25_rm3.txt
  Paramètres RM3: {'fb_terms': 10, 'fb_docs': 10, 'original_query_weight': 0.5}

Lancement de la recherche RM3...

Début recherche RM3: Modèle='bm25+RM3', Tag='baseline_long_bm25_rm3', k=1000
  Init LuceneSearcher...
  Init OK.
  Config BM25 (base)...
  Activation RM3...
  RM3 activé.


Recherche baseline_long_bm25_rm3:   0%|          | 0/150 [00:00<?, ?it/s]


  150000 lignes résultats écrites: baseline_long_bm25_rm3.txt.
Recherche RM3 terminée baseline_long_bm25_rm3. Temps: 29.97s.
  Nettoyage implicite baseline_long_bm25_rm3.

--- Exécution recherche RM3 terminée. ---

Vérification création fichier /content/ap_output/runs/baseline_long_bm25_rm3.txt...
-rw-r--r-- 1 root root 8533950 Apr  9 01:17 /content/ap_output/runs/baseline_long_bm25_rm3.txt


In [ ]:
# === Cellule 8: Évaluation Finale (Tous les Runs) ===
# Lit les fichiers Qrels, lit TOUS les fichiers de résultats (.txt) du dossier RUN_DIR,
# calcule MAP et P@10, et affiche/sauvegarde les tableaux récapitulatifs finaux.

import pandas as pd
import glob
import pytrec_eval
import os
import traceback
import re # Importer re pour parser les noms de run

# Vérifier que les chemins sont définis
try:
    QRELS_DIR
    RUN_DIR
    EVAL_DIR
except NameError:
    print("ERREUR: Variables de chemin non définies. Exécutez la cellule de configuration complète.")
    raise

print(f"Préparation Qrels depuis: {QRELS_DIR}")
qrels_files = sorted(glob.glob(os.path.join(QRELS_DIR, "qrels.*.txt")))
if not qrels_files: print(f"ATTENTION: Aucun fichier Qrels trouvé dans {QRELS_DIR}."); qrels_dict = {}
else:
    print(f"Fichiers Qrels trouvés: {qrels_files}")
    all_qrels_data = []
    for qf in qrels_files:
        try:
            # Lire le fichier qrels en spécifiant les types pour éviter les erreurs
            qrels_df = pd.read_csv(qf, sep='\s+', names=['query_id', 'unused', 'doc_id', 'relevance'],
                                   dtype={'query_id': str, 'unused': str, 'doc_id': str, 'relevance': int})
            all_qrels_data.append(qrels_df[['query_id', 'doc_id', 'relevance']])
        except Exception as e: print(f"Erreur lecture Qrels {qf}: {e}")
    if not all_qrels_data: print("ERREUR: Impossible lire données Qrels."); qrels_dict = {}
    else:
        combined_qrels_df = pd.concat(all_qrels_data, ignore_index=True)
        qrels_dict = {}
        # Convertir le DataFrame en dictionnaire attendu par pytrec_eval
        for _, row in combined_qrels_df.iterrows():
            qid, did, rel = str(row['query_id']), str(row['doc_id']), int(row['relevance'])
            if rel < 0: continue # Ignorer jugements négatifs
            if qid not in qrels_dict: qrels_dict[qid] = {}
            qrels_dict[qid][did] = rel
        print(f"Total {len(qrels_dict)} requêtes avec jugements chargées.")

# --- Évaluation des Runs ---
if not qrels_dict: print("\nAucun jugement de pertinence chargé, impossible d'évaluer.")
else:
    measures = {'map', 'P_10'} # Métriques à calculer
    evaluator = pytrec_eval.RelevanceEvaluator(qrels_dict, measures) # Initialiser l'évaluateur
    # Trouver tous les fichiers .txt dans le dossier des runs
    run_files = sorted(glob.glob(os.path.join(RUN_DIR, "*.txt")))
    print(f"\n{len(run_files)} fichiers de run à évaluer trouvés dans {RUN_DIR}.") # Devrait être 9 maintenant
    print(f"  Fichiers: {[os.path.basename(f) for f in run_files]}") # Afficher les noms

    results_summary = [] # Liste pour stocker les résultats agrégés
    if not run_files: print(f"ATTENTION: Aucun fichier de run (.txt) trouvé dans {RUN_DIR}.")
    else:
        # Boucler sur chaque fichier de run trouvé
        for run_file in run_files:
            run_name = os.path.basename(run_file)
            print(f"\n--- Évaluation: {run_name} ---")
            run_dict = {} # Dictionnaire pour stocker les résultats de ce run
            error_count = 0
            line_count = 0
            try:
                # Lire le fichier run ligne par ligne
                with open(run_file, 'r', encoding='utf-8') as f_run:
                    for line in f_run:
                        line_count += 1
                        parts = line.strip().split()
                        # Vérifier le format TREC (6 colonnes)
                        if len(parts) != 6: error_count += 1; continue
                        qid, _, did, _, score, _ = parts # Extraire les infos utiles
                        try: score = float(score) # Convertir le score en float
                        except ValueError: error_count += 1; continue
                        qid = str(qid) # Assurer que qid est une chaîne
                        # Stocker le score pour ce document et cette requête
                        if qid not in run_dict: run_dict[qid] = {}
                        run_dict[qid][did] = score
                if error_count > 0: print(f"  Avertissement: {error_count} lignes mal formatées ignorées sur {line_count} lignes.")

                # Filtrer le run pour ne garder que les requêtes présentes dans les Qrels
                filtered_run_dict = {qid: docs for qid, docs in run_dict.items() if qid in qrels_dict}
                ignored_q = len(run_dict) - len(filtered_run_dict)
                if ignored_q > 0: print(f"  Avertissement: {ignored_q} requêtes run ignorées (absentes Qrels).") # Normal
                if not filtered_run_dict: print("  Erreur: Aucune requête ne correspond aux Qrels."); continue

                # Évaluer le run filtré avec pytrec_eval
                eval_results = evaluator.evaluate(filtered_run_dict)
                # Calculer les moyennes des métriques sur toutes les requêtes évaluées
                all_maps = [q_res.get("map", 0) for q_res in eval_results.values()]
                all_p10s = [q_res.get("P_10", 0) for q_res in eval_results.values()]
                avg_map = sum(all_maps) / len(all_maps) if all_maps else 0
                avg_p10 = sum(all_p10s) / len(all_p10s) if all_p10s else 0

                # Afficher les résultats moyens pour ce run
                print(f"  MAP: {avg_map:.4f}")
                print(f"  P@10: {avg_p10:.4f}")

                # Extraire les informations du nom de fichier pour le résumé
                run_name_parts = run_name.replace('.txt','')
                parts = run_name_parts.split('_')
                if len(parts) >= 3:
                    index_type_raw = parts[0] # baseline ou preproc
                    query_type = parts[1] # short ou long
                    # Gérer les suffixes _stem et _rm3
                    model_parts = parts[2:]
                    model_suffix = ""
                    is_stem = False
                    # Vérifier si le dernier élément est 'rm3'
                    if model_parts and model_parts[-1] == 'rm3':
                        model_suffix += "+RM3"
                        model_parts = model_parts[:-1] # Enlever 'rm3'
                    # Vérifier si le dernier élément (restant) est 'stem'
                    if model_parts and model_parts[-1] == 'stem':
                        is_stem = True
                        model_parts = model_parts[:-1] # Enlever 'stem'

                    model_type = "_".join(model_parts) # Devrait être BM25 ou QLD

                    # Utiliser un nom d'index plus clair pour l'affichage
                    if index_type_raw == "preproc":
                         display_index_type = "Preprocessed (Stem)" if is_stem else "Preprocessed (??)" # Devrait être Stem ici
                    else:
                         display_index_type = "Baseline"

                    # Ajouter les résultats au résumé
                    results_summary.append({
                        "Run Name": run_name, "Index": display_index_type,
                        "Query Type": query_type.capitalize(),
                        "Weighting Scheme": model_type.upper() + model_suffix, # Ex: BM25, QLD, BM25+RM3
                        "MAP": avg_map, "P@10": avg_p10
                    })
                else: print(f"  Avertissement: Impossible parser nom run '{run_name}'.")

            except FileNotFoundError: print(f"  Erreur: Fichier run non trouvé: {run_file}")
            except Exception as e: print(f"  Erreur évaluation {run_name}: {e}"); traceback.print_exc()

        # Afficher et sauvegarder le résumé final
        if results_summary:
            print("\n\n=== Tableau Récapitulatif Final (Tous les Runs) ===")
            results_df = pd.DataFrame(results_summary)
            # Trier pour une meilleure lisibilité
            results_df = results_df.sort_values(by=["Index", "Query Type", "Weighting Scheme"])

            # Afficher le DataFrame complet
            print("\n--- Résultats Complets Finaux ---")
            print(results_df.to_markdown(index=False, floatfmt=".4f"))

            # Essayer d'afficher les tableaux pivots
            try:
                pivot_map = results_df.pivot_table(index=['Query Type', 'Weighting Scheme'], columns='Index', values='MAP')
                print("\n--- MAP Final (Tableau Pivot) ---")
                print(pivot_map.to_markdown(floatfmt=".4f"))
            except Exception as e_pivot: print(f"\n(Erreur création tableau pivot MAP: {e_pivot})")

            try:
                pivot_p10 = results_df.pivot_table(index=['Query Type', 'Weighting Scheme'], columns='Index', values='P@10')
                print("\n--- P@10 Final (Tableau Pivot) ---")
                print(pivot_p10.to_markdown(floatfmt=".4f"))
            except Exception as e_pivot: print(f"\n(Erreur création tableau pivot P@10: {e_pivot})")

            # Sauvegarder le DataFrame complet final
            summary_file_path = os.path.join(EVAL_DIR, "evaluation_summary_final.csv") # Nom final
            try:
                 results_df.to_csv(summary_file_path, index=False)
                 print(f"\nTableau récapitulatif complet sauvegardé: {summary_file_path}")
            except Exception as e_save: print(f"\nErreur sauvegarde résumé: {e_save}")
        else: print("\nAucun résultat d'évaluation à afficher.")



Préparation Qrels depuis: /content/drive/My Drive/Projet_RI/TREC/jugements de pertinence
Fichiers Qrels trouvés: ['/content/drive/My Drive/Projet_RI/TREC/jugements de pertinence/qrels.1-50.AP8890.txt', '/content/drive/My Drive/Projet_RI/TREC/jugements de pertinence/qrels.101-150.AP8890.txt', '/content/drive/My Drive/Projet_RI/TREC/jugements de pertinence/qrels.51-100.AP8890.txt']
Total 150 requêtes avec jugements chargées.

9 fichiers de run à évaluer trouvés dans /content/ap_output/runs.
  Fichiers: ['baseline_long_bm25.txt', 'baseline_long_bm25_rm3.txt', 'baseline_long_qld.txt', 'baseline_short_bm25.txt', 'baseline_short_qld.txt', 'preproc_long_bm25_stem.txt', 'preproc_long_qld_stem.txt', 'preproc_short_bm25_stem.txt', 'preproc_short_qld_stem.txt']

--- Évaluation: baseline_long_bm25.txt ---
  Avertissement: 99 requêtes run ignorées (absentes Qrels).
  MAP: 0.2205
  P@10: 0.4765

--- Évaluation: baseline_long_bm25_rm3.txt ---
  Avertissement: 99 requêtes run ignorées (absentes Qrels)

In [ ]:
# === Sauvegarde Finale (Stemming+RM3) vers Nouveau Dossier Drive ===
import os
import subprocess
import time
from datetime import datetime # Pour ajouter une date au nom du dossier

# Chemin de base du projet sur Drive (vérifiez qu'il est correct)
try: DRIVE_PROJECT_PATH
except NameError: print("ERREUR: DRIVE_PROJECT_PATH non défini."); raise

# Dossier source dans Colab à sauvegarder
SOURCE_DIR_TO_SAVE = "/content/ap_output"

# --- Création d'un NOUVEAU nom de dossier cible sur Drive ---
# Ajouter une date ou une description pour le distinguer
timestamp = datetime.now().strftime("%Y%m%d") # Format YYYYMMDD
TARGET_BACKUP_DIR_ON_DRIVE = os.path.join(DRIVE_PROJECT_PATH, f"colab_output_backup_stem_rm3_{timestamp}")
# Ou un nom plus simple si vous préférez:
# TARGET_BACKUP_DIR_ON_DRIVE = os.path.join(DRIVE_PROJECT_PATH, "colab_output_backup_stemming_final")

print(f"Source à sauvegarder : {SOURCE_DIR_TO_SAVE}")
print(f"Cible sur Drive (Nouveau Dossier) : {TARGET_BACKUP_DIR_ON_DRIVE}")

# Vérifier si le dossier source existe
if os.path.exists(SOURCE_DIR_TO_SAVE):
    # Créer le dossier cible sur Drive (ne devrait pas exister)
    print(f"Création du dossier cible: {TARGET_BACKUP_DIR_ON_DRIVE}")
    # Utiliser -p pour ne pas échouer s'il existe par hasard et créer les parents si besoin
    os.makedirs(TARGET_BACKUP_DIR_ON_DRIVE, exist_ok=True)

    print("\nCopie des fichiers vers Google Drive en cours... (Cela peut prendre plusieurs minutes)")
    # Utiliser cp -r (récursif) et -v (verbeux). Pas besoin de -u car c'est un nouveau dossier.
    copy_cmd = f"cp -r -v '{SOURCE_DIR_TO_SAVE}/.' '{TARGET_BACKUP_DIR_ON_DRIVE}/'"
    try:
        process = subprocess.run(copy_cmd, shell=True, check=True, capture_output=True, text=True, timeout=900) # Timeout 15 minutes
        print("... (Sortie de la copie) ...")
        stderr_lines = process.stderr.splitlines()
        print("\nExtrait de la fin de la sortie STDERR (fichiers copiés):")
        for line in stderr_lines[-20:]: print(line) # Afficher les 20 dernières lignes copiées

        print("\nSauvegarde terminée avec succès !")
        print(f"Le contenu de {SOURCE_DIR_TO_SAVE} a été copié dans le NOUVEAU dossier {TARGET_BACKUP_DIR_ON_DRIVE}")
        # Vérifier rapidement si les dossiers runs et eval existent dans la sauvegarde
        print("\nVérification de la sauvegarde sur Drive (partiel):")
        print(f"Contenu de {TARGET_BACKUP_DIR_ON_DRIVE}/runs :")
        !ls -l "{TARGET_BACKUP_DIR_ON_DRIVE}/runs" | head -n 10 # Devrait montrer 9 fichiers .txt
        print(f"\nContenu de {TARGET_BACKUP_DIR_ON_DRIVE}/eval :")
        !ls -l "{TARGET_BACKUP_DIR_ON_DRIVE}/eval" # Devrait montrer evaluation_summary_final.csv
    except subprocess.CalledProcessError as e:
         print(f"\nERREUR lors de la sauvegarde (code {e.returncode}).")
         print("STDERR:", e.stderr)
    except Exception as e:
        print(f"\nERREUR inattendue lors de la sauvegarde: {e}")
else:
    print(f"Le dossier source {SOURCE_DIR_TO_SAVE} n'existe pas, aucune sauvegarde effectuée.")



Source à sauvegarder : /content/ap_output
Cible sur Drive (Nouveau Dossier) : /content/drive/My Drive/Projet_RI/TREC/colab_output_backup_stem_rm3_20250409
Création du dossier cible: /content/drive/My Drive/Projet_RI/TREC/colab_output_backup_stem_rm3_20250409

Copie des fichiers vers Google Drive en cours... (Cela peut prendre plusieurs minutes)
... (Sortie de la copie) ...

Extrait de la fin de la sortie STDERR (fichiers copiés):

Sauvegarde terminée avec succès !
Le contenu de /content/ap_output a été copié dans le NOUVEAU dossier /content/drive/My Drive/Projet_RI/TREC/colab_output_backup_stem_rm3_20250409

Vérification de la sauvegarde sur Drive (partiel):
Contenu de /content/drive/My Drive/Projet_RI/TREC/colab_output_backup_stem_rm3_20250409/runs :
total 73108
-rw------- 1 root root 8533950 Apr  9 01:31 baseline_long_bm25_rm3.txt
-rw------- 1 root root 8040695 Apr  9 01:31 baseline_long_bm25.txt
-rw------- 1 root root 7876155 Apr  9 01:31 baseline_long_qld.txt
-rw------- 1 root root